In [ ]:
import sys
import os

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import datetime
import copy
import numdifftools
import statsmodels as sm
from statsmodels.stats.diagnostic import acorr_ljungbox

import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import ticker as mticker


# Local files
__root__ = '../../'
__pydirs__ = ['py', 'settings']
module_path = os.path.abspath(os.path.join(__root__[:-1]))
if module_path not in sys.path:
    for pydir in __pydirs__:
        sys.path.append(f"{module_path}/{pydir}/")

import T23_setup

import log_ret

import misc_functions as misc_fns
import plot_functions as plot_fns
import dist_functions as dist_fns
import hawkes_functions as hwks_fns
import log_ret_functions as lr_fns
import cvol_functions as cvol_fns
import greek_roman as GR
import threshold as thresh

import misc_functions_nb as misc_fns_nb

In [ ]:
# Set seed
np.random.seed(3791)

In [ ]:
plot_fns.plt_setup(plt, SMALL_SIZE=6.5, MEDIUM_SIZE=6.5, BIGGER_SIZE=8, font_family='serif', use_tex=False)
cols = plot_fns.calc_cols()
cols_CI = plot_fns.calc_cols_CI()
cols_tail = plot_fns.calc_cols_tail(rows=True)

In [ ]:
r_r = f"r"

resid_r = f"\\tilde{{{r_r}}}"
sim_r = f"\\acute{{{r_r}}}"

tilde_t_cont = "\\Lambda"
tilde_t_disc = "\\Pi"

In [ ]:
Tmisc = T23_setup.T23_misc()
fstmp = T23_setup.T23_Cfstmp(root=__root__, n=2, title='T23')
fstmp.calc_fnsubs_default()

In [ ]:
fix_a_lambda = True
_setup_T23 = T23_setup.setup_T23(__C__='Cm2', __fit__=False, __vares__=False,
                                    set_cv=False, set_tt=True,
                                    tt_fix_a_lambda=fix_a_lambda,
                                    all_end=pd.Timestamp('2023-09-16'),
                                    )

In [ ]:
_setup_T23['tt_a_u']

In [ ]:
B20 = log_ret.log_ret(dir_base=__root__, ticks=_setup_T23['ticks'])
B20.set_tk(**misc_fns.dict_key_rm(_setup_T23, keys_rm='ticks'))

In [ ]:
cv_mp_tick_time = B20.set_cv_mp_tick(set_am_stats=True, set_eval=False, append_fit_cv_w=True, append_vrs_cv_w=False, write_tick=False)

In [ ]:
B20.set_fit_tt_mp_tick(write_tick=False,
                        a_u_rng=[1], tt_fix_a_lambda=fix_a_lambda, tt_read_params=True, tt_fit_if_need=False, append_fit_tt_w=False)

In [ ]:
tick='SPX'
mode='bi'
mark='mark'
B20.tk[tick].tt.tt[mode][mark]._parameters.a_u.get_value()

In [ ]:
tails = [f"L", f"R"]
signs = np.array([-1,1])


T_year = 252


str_dp=1
str_exp_plus=True
str_exp_zeros=2


tail_fit_setup = {
                    'u': np.outer([-1,1], [0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.2]),
                    'a_u': np.outer([1,1], _setup_T23['tt_a_u']),
                }




for tick in B20.tk:
    B20.tk[tick]._T = {period: B20.tk[tick].pd.lr["p"][period].values.size for period in ['train', 'all']}
    B20.tk[tick]._N = {period: np.array([[np.sum(signs[i]*B20.tk[tick].pd.lr["p"][period].values > signs[i]*tail_fit_setup['u'][i,j]) for j in range(tail_fit_setup['u'].shape[1])] for i in range(signs.size)]) for period in ['train', 'all']}
    for val in ['T', 'N']:
        B20.tk[tick].__dict__[f'_{val}']['forecast'] = B20.tk[tick].__dict__[f'_{val}']['all'] - B20.tk[tick].__dict__[f'_{val}']['train']



    B20.tk[tick]._stat_dist_fit = dict()
    for dist in ['norm', 't']:
        B20.tk[tick]._stat_dist_fit[f"{dist}_ML"] = dist_fns.calc_scipy_dist_est_se_log_L(data=B20.tk[tick].pd.lr["p"]["train"].values, dist=dist)
    Q_rng = [0.25, 0.75]
    B20.tk[tick]._stat_dist_fit[f"norm_Q"] = dist_fns.calc_scipy_dist_est_se_log_L(data=B20.tk[tick].pd.lr["p"]["train"].values, dist='norm', est=np.array([np.median(B20.tk[tick].pd.lr["p"]["train"].values),
                                                                                                                                        (np.diff(np.quantile(B20.tk[tick].pd.lr["p"]["train"].values, Q_rng))/np.diff(scipy.stats.norm().ppf(Q_rng)))[0]]))
    
    meanstd = B20.tk[tick]._stat_dist_fit[f"norm_ML"]['est']
    B20.tk[tick]._stat_dist_fit["t_ML_f"] = dist_fns.calc_scipy_dist_est_se_log_L(data=B20.tk[tick].pd.lr["p"]["train"].values, dist='t', est=np.array([meanstd[0], meanstd[1],
                                                                                                                                                scipy.stats.t.fit((B20.tk[tick].pd.lr["p"]["train"].values-meanstd[0])/meanstd[1], floc=0, fscale=1)[0]]))
    
    B20.tk[tick]._stat_dist = dict()
    B20.tk[tick]._stat_dist_symb = dict()
    B20.tk[tick]._stat_dist_symb_fit = dict()
    B20.tk[tick]._stat_dist_symb_mod = dict()
    for dist in B20.tk[tick]._stat_dist_fit:
        B20.tk[tick]._stat_dist[dist] = scipy.stats.__getattribute__(dist[:dist.find("_")])(*B20.tk[tick]._stat_dist_fit[dist]['est'])
        B20.tk[tick]._stat_dist_symb[dist] = dist_fns.get_scipy_dist_symb(dist[:dist.find('_')])
        B20.tk[tick]._stat_dist_symb_fit[dist] = dist_fns.get_scipy_dist_symb(dist[:dist.find('_')])
        B20.tk[tick]._stat_dist_symb_mod[dist] = dist_fns.get_scipy_dist_symb(dist[:dist.find('_')])

    for dist in ['norm', 't']:
        #B20.tk[tick]._stat_dist_symb_fit[f"{dist}_ML"] = B20.tk[tick]._stat_dist_symb_fit[f"{dist}_ML"] + f"_{{\\mathrm{{ML}}}}"
        #B20.tk[tick]._stat_dist_symb_fit[f"{dist}_ML"] = f"\\hat{{" + B20.tk[tick]._stat_dist_symb_fit[f"{dist}_ML"] + f"}}_{{W}}"
        B20.tk[tick]._stat_dist_symb_fit[f"{dist}_ML"] = f"\\hat{{{B20.tk[tick]._stat_dist_symb_fit[f'{dist}_ML']}}}_{{W}}"
        B20.tk[tick]._stat_dist_symb_mod[f"{dist}_ML"] = f"{cvol_fns.calc_cv_vol_symb(val='uv')}^{{{B20.tk[tick]._stat_dist_symb[f'{dist}_ML']}}}"
    
    B20.tk[tick]._stat_dist_symb_fit[f"norm_Q"] = f"\\hat{{{B20.tk[tick]._stat_dist_symb_fit[f'norm_Q']}}}_{{Q}}"
    #B20.tk[tick]._stat_dist_symb_fit[f"t_ML_f"] = B20.tk[tick]._stat_dist_symb_fit[f"t_ML_f"] + f"_{{\\mathrm{{ML}}}}^{{f}}"
    B20.tk[tick]._stat_dist_symb_fit[f"t_ML_f"] = f"\\hat{{{B20.tk[tick]._stat_dist_symb_fit[f't_ML_f']}}}_{{W}}^{{f}}"
    

    for mod in [f"norm_Q", f"t_ML_f"]:
        B20.tk[tick]._stat_dist_symb_mod[mod] = f"{cvol_fns.calc_cv_vol_symb(val='uv')}^{{{B20.tk[tick]._stat_dist_symb_fit[mod]}}}"

    

    B20.tk[tick]._tail_fit = {key: dist_fns.calc_tail_fit(B20.tk[tick].pd.lr["p"]["train"].values, **{key: tail_fit_setup[key]}) for key in tail_fit_setup}


    for key in B20.tk[tick]._tail_fit:
        B20.tk[tick]._tail_fit[key]['lambda']['inv_lambda_str'] = np.array([[misc_fns.scif_string((1/T_year)/B20.tk[tick]._tail_fit[key]['lambda']['a_lambda'][i,j], dp=str_dp, exp_plus=str_exp_plus, exp_zeros=str_exp_zeros) 
                                                                                for j in range(B20.tk[tick]._tail_fit[key]['lambda']['a_lambda'].shape[1])] for i in range(B20.tk[tick]._tail_fit[key]['lambda']['a_lambda'].shape[0])], dtype=object)
        
        B20.tk[tick]._tail_fit[key]['lambda']['inv_lambda_str'][np.isinf(B20.tk[tick]._tail_fit[key]['lambda']['a_lambda'])] = f"$>{misc_fns.scif_string(B20.tk[tick]._T['train']/T_year, dp=str_dp, exp_plus=str_exp_plus, exp_zeros=str_exp_zeros, app_dlrs=False)}$"


    B20.tk[tick]._inv_lambda = dict()
    B20.tk[tick]._inv_lambda["N"] = {period: (B20.tk[tick]._T[period]/T_year)/B20.tk[tick]._N[period] for period in B20.tk[tick]._T}
    
    for dist in B20.tk[tick]._stat_dist:
        #B20.tk[tick]._inv_lambda[dist] = np.array([(1/T_year)/(i + ((-1)**i)*B20.tk[tick]._stat_dist[dist].cdf(tail_fit_setup['u'][i,:])) for i in range(signs.size)])
        B20.tk[tick]._inv_lambda[dist] = np.array([np.exp(-B20.tk[tick]._stat_dist[dist].logcdf(i*2*B20.tk[tick]._stat_dist[dist].mean() + ((-1)**i)*tail_fit_setup['u'][i,:])) for i in range(tail_fit_setup['u'].shape[0])])/T_year
        
        

    B20.tk[tick]._inv_lambda_str = dict()
    for dist in  B20.tk[tick]._stat_dist:
        B20.tk[tick]._inv_lambda_str[dist] = np.array([[misc_fns.scif_string(B20.tk[tick]._inv_lambda[dist][i,j], dp=str_dp, exp_plus=str_exp_plus, exp_zeros=str_exp_zeros) for j in range(tail_fit_setup['u'].shape[1])] for i in range(signs.size)], dtype=object)

    B20.tk[tick]._inv_lambda_str["N"] = {period: np.array([[misc_fns.scif_string(B20.tk[tick]._inv_lambda["N"][period][i,j], dp=str_dp, exp_plus=str_exp_plus, exp_zeros=str_exp_zeros) for j in range(tail_fit_setup['u'].shape[1])] for i in range(signs.size)], dtype=object) for period in B20.tk[tick]._inv_lambda["N"]}
    for period in B20.tk[tick]._inv_lambda_str["N"]:
        B20.tk[tick]._inv_lambda_str["N"][period][np.isinf(B20.tk[tick]._inv_lambda["N"][period])] = f"$>{misc_fns.scif_string(B20.tk[tick]._T[period]/T_year, dp=str_dp, exp_plus=str_exp_plus, exp_zeros=str_exp_zeros, app_dlrs=False)}$"
        B20.tk[tick]._inv_lambda_str["N"][period][np.isnan(B20.tk[tick]._inv_lambda["N"][period])] = f""

In [ ]:
B20.tk[tick]._tail_fit['u']['lambda']['a_u']

In [ ]:
B20.tk[tick]._tail_fit['u']['lambda']['u']

In [ ]:
B20.tk[tick]._tail_fit['a_u']['lambda']['a_u']

In [ ]:
B20.tk[tick]._tail_fit['a_u']['lambda']['u']

In [ ]:
def print_macros(save=None, print_dir=None, do_print=None, do_write=None, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None):

    C_str = fstmp.calc_C_str()

    tab_str = []

    bounds = ['start', 'end', 'T']
    periods = ['all', 'train', 'forecast']
    a_u_key = 1
    tails = ['left', 'right']

    all_start_dates_price_0 = {tick: _setup_T23['all_start'][tick].strftime('%Y-%m-%d') for tick in _setup_T23['all_start']}


    dates_ts_latex = {tick: {period: {bound: None for bound in bounds} for period in periods} for tick in B20.tk}
    dates_T_latex = {tick: {period: None for period in periods} for tick in B20.tk} 
    T = {period: None for period in periods}


    for tick in B20.tk:
        for period in periods:
            if period=='forecast':
                dates_ts_latex[tick][period]['start'] = B20.tk[tick].pd._dates['train'][-1].strftime('%Y-%m-%d')
                dates_ts_latex[tick][period]['end'] = B20.tk[tick].pd._dates['all'][-1].strftime('%Y-%m-%d')
                T[period] = T['all'] - T['train']
            else:
                for i in range(2):
                    bound = bounds[i]
                    dates_ts_latex[tick][period][bound] = B20.tk[tick].pd._dates[period][i].strftime('%Y-%m-%d')
                T[period] = B20.tk[tick].pd.lr['p'][period].values.size

            dates_ts_latex[tick][period]['T'] = Tmisc.capp_tex_num(T[period])


    

    multi_all_start = True
    tab_str.append(f"% Dates")
    if multi_all_start:
        
        tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDatePriceAllStartALL}}{{}}")
        tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDateAllStartALL}}{{}}")
    else:
        tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDateAllStartALL}}{{{_setup_T23['all_start'].strftime('%Y-%m-%d')}}}")

    tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDateAllEndALL}}{{{_setup_T23['all_end'].strftime('%Y-%m-%d')}}}")
    if multi_all_start:
        tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDateTrainStartALL}}{{}}")
    else:
        tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDateTrainStartALL}}{{{_setup_T23['all_start'].strftime('%Y-%m-%d')}}}")
    tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDateTrainEndALL}}{{{_setup_T23['train_end'].strftime('%Y-%m-%d')}}}")
    tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDateForecastStartALL}}{{{_setup_T23['train_end'].strftime('%Y-%m-%d')}}}")
    tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDateForecastEndALL}}{{{_setup_T23['all_end'].strftime('%Y-%m-%d')}}}")
    tab_str.append('')

    for tick in B20.tk:
        tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDatePriceAllStart{tick}}}{{{all_start_dates_price_0[tick]}}}")
    for period in periods:
        for bound in bounds:
            for tick in B20.tk:
                tab_str.append(f"\\newcommand{{\\{C_str['l']}rsDate{period.capitalize()}{bound.capitalize()}{tick}}}{{{dates_ts_latex[tick][period][bound]}}}")
            tab_str.append('')



    tab_str.append(f"% Thresholds")
    for i in range(len(tails)):
        tab_str.append(f"\\newcommand{{\\{C_str['l']}rsThresh{tails[i].capitalize()}Au}}{{{B20.tk[tick]._tail_fit['a_u']['lambda']['a_u'][i,a_u_key]}}}")
        for tick in B20.tk:
            tab_str.append(f"\\newcommand{{\\{C_str['l']}rsThresh{tails[i].capitalize()}{tick}}}{{{B20.tk[tick]._tail_fit['a_u']['lambda']['u'][i,a_u_key]:+.5f}}}")
        tab_str.append("")



    if file_nm is None:
        file_nm = "macros"

    fstmp.print_str_pw(print_str=tab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=file_nm, file_ext=file_ext)


In [ ]:
print_macros(do_print=True, do_write=True, write_mode=None, file_nm=None)

In [ ]:

def print_table_exceed(self, period=None, u_s=None, metaticks=None, tick_cols=None, do_print=None, do_write=None, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, print_ftab=None, ftab_size=None):

    if print_ftab is None:
        print_ftab = True

    C_str = fstmp.calc_C_str()

    tab_str = []
    ftab_str = []


    if period is None:
        period = 'train'

    if metaticks is None:
        metaticks = [tick for tick in self.tk]
    metaticks = np.array(metaticks)

    if tick_cols is None:
        tick_cols = 1

    metaticks = np.reshape(metaticks, (metaticks.size//tick_cols,tick_cols))

    if u_s is None:
        u_s = self.tk[metaticks[0,0]]._tail_fit['u']['lambda']['u']

    chev = [f"<", f">"]
    tails = ['L', 'R']

    left_r = 0
    

    for ticks in metaticks:

        ticks_str = misc_fns.str_concatenate(ticks)

        tick_tail_strs = [[f'\\multicolumn{{5}}{{c}}{{{tick}, $\\rsT{tails[i]}$}} ' for tick in ticks] for i in range(len(tails))]
        tick_tail_str = [f"{misc_fns.str_concatenate(tick_tail_strs[i], fill_str='& & ', fill_str_void=-1)}\\\\" for i in range(len(tails))]
        tick_line_strs = [f"\\cmidrule{{{1+6*i}-{5+6*i}}}" for i in range(len(ticks))]
        tick_line_str = misc_fns.str_concatenate(tick_line_strs)


        #expect_head_tail_strs = [f"& \\multicolumn{{3}}{{c}}{{$\\mathbb{{E}}{{\\left[a_{{\\lambda}}^{{\\rsT{tails[i]}^{{-1}}}} \\middle| \\rsPr_{{t}} {chev[i]} u^{{\\rsT{tails[i]}}} \\right] / \\mathrm{{yr}}}}$}} & " for i in range(len(tails))]
        expect_head_tail_strs = [f"& \\multicolumn{{3}}{{c}}{{$a_{{\\rsMdlPrtPotPr}}^{{\\rsT{tails[i]}^{{-1}}}} \\rsUdt / \\rsUyt$}} & " for i in range(len(tails))]
        expect_head_tail_str = [misc_fns.str_concatenate([expect_head_tail_strs[i] for tick in ticks], fill_str='\n& &\n', fill_str_void=-1) + "\n\\\\" for i in range(len(tails))]
        expect_head_line_strs = [f"\\cmidrule{{{2+6*i}-{4+6*i}}}" for i in range(len(ticks))]
        expect_head_line_str =  misc_fns.str_concatenate(expect_head_line_strs)


        col_head_tail_strs = {tick: [f"$u^{{\\rsT{tails[i]}}}$ & ${cvol_fns.calc_uv_symb(dist='norm', tex=True)}$ & ${cvol_fns.calc_uv_symb(dist='t', tex=True)}$ & $\\Delta{{T}}/\\Delta{{N}}^{{\\rsT{tails[i]}}}$ & $\\Delta{{N}}^{{\\rsT{tails[i]}}}$ " for i in range(len(tails))] for tick in ticks}
        col_head_tail_str = [f"{misc_fns.str_concatenate([col_head_tail_strs[tick][i] for tick in ticks], fill_str='& & ', fill_str_void=-1)} \\\\" for i in range(len(tails))]
        col_line_str = tick_line_str


        #t_shape = np.array(np.shape(np.array(ticks_t)))
        #J = t_shape[1]

        c_str_j = 'rlllr'
        J = np.size(ticks)
        C = len(c_str_j)
        c_str = misc_fns.str_concatenate([c_str_j for j in range(J)], fill_str='c', fill_str_void=-1)
        C_J = J*(C+1)-1

        #ticks_t_shape_str = misc_fns.str_concatenate(t_shape, fill_str='x', fill_str_void=-1)
        #ticks_t_str = misc_fns.str_concatenate([misc_fns.str_concatenate(misc_fns.make_iterable_array(ticks_t[i])) for i in range(len(ticks_t))], fill_str='x', fill_str_void=-1)

        tab_macro_str = f"TABExceedInterval{ticks_str}"
        tab_mfn_str = f"\\{C_str['l']}{tab_macro_str}"


        if print_ftab:
            ftab_str = plot_fns.calc_ftab_str(tab_mfn_str=tab_mfn_str, c_str=c_str, sizes=None, size=ftab_size, ftab_str=ftab_str)

        tab_str.append(f"\\newcommand{{{tab_mfn_str}}}{{")

        for i in range(u_s.shape[0]):
            tab_str.append(tick_tail_str[i])
            tab_str.append(tick_line_str)
            tab_str.append(expect_head_tail_str[i])
            tab_str.append(expect_head_line_str)
            tab_str.append(col_head_tail_str[i])
            tab_str.append(col_line_str)

            for j in range(u_s.shape[1]):
                N_str = [Tmisc.capp_dlrs_tex_num(S=self.tk[ticks[t]]._N[period][i,j]) for t in range(len(ticks))]
                tab_str.append(misc_fns.str_concatenate([
                                    f"${u_s[i,j]:+.3f}$ & {self.tk[ticks[t]]._inv_lambda_str['norm_ML'][i,j]} & {self.tk[tick]._inv_lambda_str['t_ML'][i,j]} & {self.tk[ticks[t]]._inv_lambda_str['N'][period][i,j]} & {N_str[t]}"
                                    for t in range(len(ticks))], fill_str="\n& &\n", fill_str_void=-1))

                tab_str.append("\\\\")
            if i < u_s.shape[0]-1:
                tab_str.append(f"\\hline")

        tab_str.append(f"}}")
        tab_str.append("")

    
    if file_nm is None:
        file_nm = "exceed"

    fstmp.print_str_pw(print_str=tab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"tab_{file_nm}", file_ext=file_ext)
    if print_ftab:
        fstmp.print_str_pw(print_str=ftab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"ftab_{file_nm}", file_ext=file_ext)


In [ ]:

def print_table_exceed_flip(self, period=None, u_s=None, metaticks=None, tick_cols=None, do_print=None, do_write=None, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, print_ftab=None, ftab_size=None):


    if print_ftab is None:
        print_ftab = True

    C_str = fstmp.calc_C_str()

    tab_str = []
    ftab_str = []

    if period is None:
        period = 'train'

    if metaticks is None:
        metaticks = [tick for tick in self.tk]
    metaticks = np.array(metaticks)

    if tick_cols is None:
        tick_cols = 1

    metaticks = np.reshape(metaticks, (metaticks.size//tick_cols,tick_cols))

    if u_s is None:
        u_s = self.tk[metaticks[0,0]]._tail_fit['u']['lambda']['u']

    chev = [f"<", f">"]
    tails = ['L', 'R']

    left_r = 0

    

    for ticks in metaticks:

        ticks_str = misc_fns.str_concatenate(ticks)

        tick_tick_strs = {tick: misc_fns.str_concatenate([f'\\multicolumn{{5}}{{c}}{{{tick}, $\\rsT{tails[i]}$}} ' for i in range(len(tails))], fill_str='& & ', fill_str_void=-1)+f"\\\\" for tick in ticks}
        tick_line_strs = [f"\\cmidrule{{{1+6*i}-{5+6*i}}}" for i in range(u_s.shape[0])]
        tick_line_str = misc_fns.str_concatenate(tick_line_strs)


        #expect_head_tail_str = misc_fns.str_concatenate([f"& \\multicolumn{{3}}{{c}}{{$\\mathbb{{E}}{{\\left[a_{{\\lambda}}^{{\\rsT{tails[i]}^{{-1}}}} \\middle| \\rsPr_{{t}} {chev[i]} u^{{\\rsT{tails[i]}}} \\right] / \\mathrm{{yr}}}}$}} & " for i in range(len(tails))], fill_str='& & ', fill_str_void=-1)+f"\\\\"
        expect_head_tail_str = misc_fns.str_concatenate([f"& \\multicolumn{{3}}{{c}}{{$a_{{\\rsMdlPrtPotPr}}^{{\\rsT{tails[i]}^{{-1}}}} \\rsUdt / \\rsUyt$}} & " for i in range(len(tails))], fill_str='\n& &\n', fill_str_void=-1)+f"\n\\\\"
        expect_head_line_strs = [f"\\cmidrule{{{2+6*i}-{4+6*i}}}" for i in range(u_s.shape[0])]
        expect_head_line_str =  misc_fns.str_concatenate(expect_head_line_strs)


        col_head_tail_str = {tick: misc_fns.str_concatenate([f"$u^{{\\rsT{tails[i]}}}$ & ${cvol_fns.calc_uv_symb(dist='norm', tex=True)}$ & ${cvol_fns.calc_uv_symb(dist='t', tex=True)}$ & $\\Delta{{T}}/\\Delta{{N}}^{{\\rsT{tails[i]}}}$ & $\\Delta{{N}}^{{\\rsT{tails[i]}}}$ " for i in range(len(tails))], fill_str='& & ', fill_str_void=-1)+f"\\\\" for tick in ticks}
        col_line_str = tick_line_str



        #t_shape = np.array(np.shape(np.array(ticks_t)))
        #J = t_shape[1]

        c_str_j = 'rlllr'
        J = 2
        C = len(c_str_j)
        c_str = misc_fns.str_concatenate([c_str_j for j in range(J)], fill_str='c', fill_str_void=-1)
        C_J = J*(C+1)-1



        #ticks_t_shape_str = misc_fns.str_concatenate(t_shape, fill_str='x', fill_str_void=-1)
        #ticks_t_str = misc_fns.str_concatenate([misc_fns.str_concatenate(misc_fns.make_iterable_array(ticks_t[i])) for i in range(len(ticks_t))], fill_str='x', fill_str_void=-1)

        tab_macro_str = f"TABExceedInterval{ticks_str}"
        tab_mfn_str = f"\\{C_str['l']}{tab_macro_str}"

        if print_ftab:
            ftab_str = plot_fns.calc_ftab_str(tab_mfn_str=tab_mfn_str, c_str=c_str, sizes=None, size=ftab_size, ftab_str=ftab_str)


        tab_str.append(f"\\newcommand{{{tab_mfn_str}}}{{")
        for tick in ticks:
            tab_str.append(tick_tick_strs[tick])
            tab_str.append(tick_line_str)
            tab_str.append(expect_head_tail_str)
            tab_str.append(expect_head_line_str)
            tab_str.append(col_head_tail_str[tick])
            tab_str.append(col_line_str)

            for j in range(u_s.shape[1]):
                N_str = [Tmisc.capp_dlrs_tex_num(S=self.tk[tick]._N[period][i,j]) for i in range(u_s.shape[0])]
                tab_str.append(misc_fns.str_concatenate([
                                    f"${u_s[i,j]:+.3f}$ & {self.tk[tick]._inv_lambda_str['norm_ML'][i,j]} & {self.tk[tick]._inv_lambda_str['t_ML'][i,j]} & {self.tk[tick]._inv_lambda_str['N'][period][i,j]} & {N_str[i]}"
                                     for i in range(u_s.shape[0])], fill_str="\n& &\n", fill_str_void=-1)
                                )
                tab_str.append("\\\\")
            if tick!=ticks[-1]:
                tab_str.append(f"\\hline")

        tab_str.append(f"}}")
        tab_str.append("")

    if file_nm is None:
        file_nm = "exceed"

    fstmp.print_str_pw(print_str=tab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"tab_{file_nm}", file_ext=file_ext)
    if print_ftab:
        fstmp.print_str_pw(print_str=ftab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"ftab_{file_nm}", file_ext=file_ext)


In [ ]:
print_table_exceed(self=B20, tick_cols=1, do_print=True, do_write=True, write_mode='w', file_nm=None)
#print_table_exceed(self=B20, tick_cols=2, do_print=True, do_write=True, write_mode='a', file_nm=None)
print_table_exceed_flip(self=B20, metaticks=['SPX', 'DJI', 'NKX', 'DAX', 'CAC', 'HSI'], tick_cols=3, do_print=True, do_write=True, write_mode='a', file_nm=None, ftab_size='script')

In [ ]:
dists = ["norm_ML", "norm_Q", "t_ML"]
#dists_cols = [cols[3], cols[1], cols[5]]

dists_cols = [cols[9], cols[5], cols[0]]

dists_select = [0,2]

ms = 9
marker = 'D'


for tick in B20.tk:

    fig_norm_qq = plot_fns.rc_plot_setup(name=f'{tick}_r_qq_base')


    data = copy.deepcopy(B20.tk[tick].pd.lr['p']['train'])


    fig_norm_qq_quantiles = (0.5+np.array(range(data.values.size)))/data.values.size
    fig_norm_qq_x = scipy.stats.norm().ppf(fig_norm_qq_quantiles)
    fig_norm_qq_x_ls = np.linspace(fig_norm_qq_x[0], fig_norm_qq_x[-1], 201)



    fig_norm_qq["ax"][0][0].scatter(fig_norm_qq_x, np.sort(data.values), color=cols[7], marker=marker, s=ms, lw=0)


    #fig_norm_qq["ax"][0][0].plot(fig_norm_qq_x[[0,-1]], normal_mean.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color='red', linewidth=1.5)
    #fig_norm_qq["ax"][0][0].plot(fig_norm_qq_x[[0,-1]], normal_median.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color="orange", linewidth=1.5) 

    for k in dists_select:
        fig_norm_qq["ax"][0][0].plot(fig_norm_qq_x_ls, B20.tk[tick]._stat_dist[dists[k]].ppf(scipy.stats.norm().cdf(fig_norm_qq_x_ls)), color=dists_cols[k], linewidth=1.5)

    
    #fig_norm_qq["ax"][0][0].plot(fig_norm_qq_x_ls, B20.tk[tick]._stat_dist['t_ML_f'].ppf(scipy.stats.norm().cdf(fig_norm_qq_x_ls)), color=cols[5], linewidth=1.5)


    #fig_norm_qq["ax"][0][0].set_ylabel(f"{tick} $r_t$")
    fig_norm_qq["ax"][0][0].set_ylabel(f"$r_t$")
    fig_norm_qq["ax"][0][0].set_xlabel(f"Normal quantiles")


    fig_norm_qq["ax"][0][0].set_xlim(4.25*np.array([-1,1]))
    fig_norm_qq["ax"][0][0].set_ylim([-0.25, 0.125])
    #fig_norm_qq["ax"][0][0].set_ylim(0.25*np.array([-1,1]))

    if False:     
        u_q = np.array(range(2)) + np.array([1,-1])*B20.tk[tick].tt.tt["bi"]["mark"]._parameters.a_lambda.get_value()
        if True:
            u_y = B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value()
            u_x = (B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value() - normal_mean.mean())/normal_mean.std()
        else:
            u_y = normal_mean.ppf(u_q)
            u_x = scipy.stats.norm().ppf(u_q)
        for p in range(2):
            fig_norm_qq["ax"][0][0].plot(np.repeat(u_x[p], 2), np.array([u_y[p], ((-1)**(1+p))*5]), linewidth=0.5, linestyle="--", color="black")
            fig_norm_qq["ax"][0][0].plot(np.array([u_x[p], ((-1)**(1+p))*5]), np.repeat(u_y[p], 2), linewidth=0.5, linestyle="--", color="black")





    xy_box = hwks_fns.annotate_model_Hawkes(ax=fig_norm_qq["ax"][0][0], 
                                            #tick=tick, series="r_{{t}}", plot_tick=True, 
                                            str_tick=tick, plot_tick=True, 
                                            an_h='left', an_v='top', 
                                            an_x0=0.015, an_y0=0.03, flip_x0ry0t=True, an_mode=fstmp.get_an_mode())

    fig_norm_qq["ax"][0][0].legend([fig_norm_qq["ax"][0][0].get_children()[p] for p in range(1+len(dists_select))], np.concatenate([[f"$r_{{t}}$"], [f"${B20.tk[tick]._stat_dist_symb_fit[dists[k]]}$" for k in dists_select]]), loc="lower right",
                                   bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box], along_x=False, along_y=True)
                                   )


    fig_norm_qq["fig"].set_size_inches(5.7, 3)
    fig_norm_qq["fig"].tight_layout()

    fstmp.fig_export(fig_norm_qq, height=3, width=5.7, formats=['png', 'pdf'], dpi=fstmp.get_dpi())


In [ ]:
dists = ["norm_ML", "norm_Q", "t_ML"]
#dists_cols = [cols[3], cols[1], cols[5]]

dists_cols = [cols[9], cols[5], cols[0]]

dists_select = [0,2]

ms = 9
marker = 'D'

ticks = np.transpose([
                ["SPX", "DJI", "NKX"],
                ["DAX", "CAC", "HSI"],
            ])


I_max = np.shape(ticks)[0]
J_max = np.shape(ticks)[1]


fig_norm_qq = plot_fns.rc_plot_setup(name=f'TICK_r_qq_base', nrows=I_max, ncols=J_max)

for i in range(I_max):
    for j in range(J_max):

        tick = ticks[i][j]

        data = copy.deepcopy(B20.tk[tick].pd.lr['p']['train'])


        fig_norm_qq_quantiles = (0.5+np.array(range(data.values.size)))/data.values.size
        fig_norm_qq_x = scipy.stats.norm().ppf(fig_norm_qq_quantiles)
        fig_norm_qq_x_ls = np.linspace(fig_norm_qq_x[0], fig_norm_qq_x[-1], 201)



        fig_norm_qq["ax"][i][j].scatter(fig_norm_qq_x, np.sort(data.values), color=cols[7], marker=marker, s=ms, lw=0)


        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x[[0,-1]], normal_mean.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color='red', linewidth=1.5)
        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x[[0,-1]], normal_median.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color="orange", linewidth=1.5) 

        for k in dists_select:
            fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x_ls, B20.tk[tick]._stat_dist[dists[k]].ppf(scipy.stats.norm().cdf(fig_norm_qq_x_ls)), color=dists_cols[k], linewidth=1.5)


        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x_ls, B20.tk[tick]._stat_dist['t_ML_f'].ppf(scipy.stats.norm().cdf(fig_norm_qq_x_ls)), color=cols[5], linewidth=1.5)


        #fig_norm_qq["ax"][i][j].set_ylabel(f"{tick} $r_t$")
        


        fig_norm_qq["ax"][i][j].set_xlim(4.25*np.array([-1,1]))
        fig_norm_qq["ax"][i][j].set_ylim([-0.25, 0.125])
        #fig_norm_qq["ax"][i][j].set_ylim(0.25*np.array([-1,1]))

        if False:     
            u_q = np.array(range(2)) + np.array([1,-1])*B20.tk[tick].tt.tt["bi"]["mark"]._parameters.a_lambda.get_value()
            if True:
                u_y = B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value()
                u_x = (B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value() - normal_mean.mean())/normal_mean.std()
            else:
                u_y = normal_mean.ppf(u_q)
                u_x = scipy.stats.norm().ppf(u_q)
            for p in range(2):
                fig_norm_qq["ax"][i][j].plot(np.repeat(u_x[p], 2), np.array([u_y[p], ((-1)**(1+p))*5]), linewidth=0.5, linestyle="--", color="black")
                fig_norm_qq["ax"][i][j].plot(np.array([u_x[p], ((-1)**(1+p))*5]), np.repeat(u_y[p], 2), linewidth=0.5, linestyle="--", color="black")





        xy_box = hwks_fns.annotate_model_Hawkes(ax=fig_norm_qq["ax"][i][j], 
                                                #tick=tick, series="r_{{t}}", plot_tick=True, 
                                                str_tick=tick, plot_tick=True, 
                                                an_h='left', an_v='top', 
                                                an_x0=0.015, an_y0=0.03, flip_x0ry0t=True, an_mode=fstmp.get_an_mode())

        if False:
            fig_norm_qq["ax"][i][j].legend([fig_norm_qq["ax"][i][j].get_children()[p] for p in range(1+len(dists_select))], np.concatenate([[f"$r_{{t}}$"], [f"$\\hat{{{B20.tk[tick]._stat_dist_symb_fit[dists[k]]}}}^{{\\mathrm{{{tick}}}}}$" for k in dists_select]]), loc="lower right",
                                            bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box], along_x=False, along_y=True)
                                            )
        else:
            if i==0 and j==0:
                fig_norm_qq["ax"][i][j].legend([fig_norm_qq["ax"][i][j].get_children()[p] for p in range(1+len(dists_select))], np.concatenate([[f"$r_{{t}}$"], [f"${B20.tk[tick]._stat_dist_symb_fit[dists[k]]}$" for k in dists_select]]), loc="lower right",
                                                bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box], along_x=False, along_y=True)
                                                )
        
        fig_norm_qq["ax"][i][j].tick_params(labelleft=(j==0), labelright=False, labelbottom=(i==I_max-1), labeltop=False)
        if j==0:
            fig_norm_qq["ax"][i][j].set_ylabel(f"$r_t$")
        if i==I_max-1:
            fig_norm_qq["ax"][i][j].set_xlabel(f"Normal quantiles")


fig_norm_qq["fig"].set_size_inches(5.7, 7)
fig_norm_qq["fig"].tight_layout()

fstmp.fig_export(fig_norm_qq, height=7, width=5.7, formats=['png', 'pdf'], dpi=fstmp.get_dpi())


In [ ]:
ticks_lr = [
                ["SPX", "DJI", "NKX"],
                ["DAX", "CAC", "HSI"],
            ]

xlims = [
            ["1927-12", "2023-06"],
            ["1959-09", "2023-06"],
        ]

ylims = [
            [-0.25,0.25],
            [-0.25,0.25],
        ]


for j in range(2):

    fig_lr = plot_fns.rc_plot_setup(name=f'TICK_lr_{1+j}', nrows=3)


    for i in range(3):

        tick = ticks_lr[j][i]

        dt = B20.tk[tick].pd.lr["p"]["train"].index
        x = np.reshape(np.repeat(dt,4), (dt.size, 4))

        y = np.full((dt.size,4), np.nan)
        y[:,1] = 0
        y[:,2] = B20.tk[tick].pd.lr["p"]["train"].values

        signs = np.array([((-1)**(1+k))*B20.tk[tick].pd.lr["p"]["train"].values > 0  for k in range(2)])

        signs[1,:] = 1 - signs[0,:]

        for k in range(2):
            fig_lr["ax"][i][0].plot(np.concatenate(x[signs[k],:]), np.concatenate(y[signs[k],:]), color=cols_tail[0][k], lw=0.3)

        fig_lr["ax"][i][0].set_xlim(xlims[j])
        fig_lr["ax"][i][0].set_ylim(ylims[j])

        #fig_lr["ax"][i][0].set_ylabel(f"$r_{{t}}$ [{tick}]")
        fig_lr["ax"][i][0].set_ylabel(f"$r_{{t}}$")

        
        xy_box = hwks_fns.annotate_model_Hawkes(ax=fig_lr["ax"][i][0], 
                                    #tick=tick, series=f"r_{{t}}", plot_tick=True, 
                                    str_tick=tick, plot_tick=True, 
                                    #str_mod=f"$\\Delta{{T}}={plot_fns.capp_tex_num(B20.tk[tick]._T['all'])}$", plot_mod=True,
                                    an_h='left', an_v='top', flip_x0ry0t=True,
                                    an_x0=0.015, an_y0=0.03, an_mode=fstmp.get_an_mode())
        
    #fig_lr["ax"][-1][0].set_xlabel(f"Date")

    fig_lr["ax"][0][0].tick_params(labeltop=False)


    fig_lr["fig"].set_size_inches(5.7, 8)
    fig_lr["fig"].tight_layout()

    fstmp.fig_export(fig_lr, height=8, width=5.7)


In [ ]:
tick='SPX'

vol ='GARCH'
p = 1
o = 0
q = 1
dist = 'norm'

B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist]._bdist.kwds

In [ ]:
ms = 1.5
markers = ['D']

dists = ['norm', 't']
dist_symb = {
                'norm': f'\\mathcal{{N}}',
                't': f'\\mathcal{{S}}',
            }


col_dists = [cols[9], cols[0]]


xlim = ["1996", "2024"]

ylim = 0.13*np.array([0,1])

K = 1

for tick in B20.tk:

    fig_cv = plot_fns.rc_plot_setup(name=f'{tick}_abs_cv')

    index = B20.tk[tick].pd.lr['p']['all'].index


    fig_cv['ax'][0][0].scatter(index, np.abs(B20.tk[tick].pd.lr['p']['all'].values), marker=markers[0], lw=0, s=ms, color=cols[7])

    for i in range(2):
        if Tmisc.get_sd_not_scale() and dists[i]=='t':
            nu = B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dists[i]]._bdist.kwds['df']
            factor = np.sqrt(nu/(nu-2))
        else:
            factor = 1
        for k in range(K):
            fig_cv['ax'][0][0].plot(index, factor*((-1)**k)*B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dists[i]]._bdist.kwds['scale'], lw=0.5, color=col_dists[i])



    fig_cv['ax'][0][0].set_xlim(xlim)
    fig_cv['ax'][0][0].set_ylim(ylim)


    fig_cv['ax'][0][0].set_xlabel(f"Date")
    fig_cv['ax'][0][0].set_ylabel(f"$\\left|r_{{t}}\\right|$")


    xy_box = hwks_fns.annotate_model_Hawkes(ax=fig_cv['ax'][0][0], 
                                            #tick=tick, series="r_{{t}}", plot_tick=True, 
                                            str_tick=tick, plot_tick=True, 
                                            an_h='left', an_v='top', 
                                            an_x0=0.015, an_y0=0.025, flip_x0ry0t=True, an_mode=fstmp.get_an_mode())



    fig_cv['ax'][0][0].legend([fig_cv['ax'][0][0].get_children()[k] for k in [0,1*K,2*K]],
                                    np.concatenate([[f"$\\left|r_{{t}}\\right|$"], [f"$\\hat{{\\sigma}}_{{t}} | {cvol_fns.calc_cv_symb(vol='GARCH', o=0, dist=d)}$" for d in dists]]),
                                    loc="upper left",
                                    bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box], along_x=False, along_y=True))



    fig_cv['fig'].set_size_inches(5.7,2.5)
    fig_cv['fig'].tight_layout()

    fstmp.fig_export(fig_cv, height=2.5, formats=['pdf', 'png'], dpi=fstmp.get_dpi())

In [ ]:
ms = 1.5
markers = ['D']

dists = ['norm', 't']
dist_symb = {
                'norm': f'\\mathcal{{N}}',
                't': f'\\mathcal{{S}}',
            }


col_dists = [cols[9], cols[0]]


xlim = ["1996", "2024"]

ylim = 0.13*np.array([0,1])

S = 1


ticks = np.transpose([
                ["SPX", "DJI", "NKX"],
                ["DAX", "CAC", "HSI"],
            ])

I_max = np.shape(ticks)[0]
J_max = np.shape(ticks)[1]


fig_cv = plot_fns.rc_plot_setup(name=f'TICK_abs_cv', nrows=I_max, ncols=J_max)

for i in range(I_max):
    for j in range(J_max):
        tick = ticks[i][j]

        index = B20.tk[tick].pd.lr['p']['all'].index

        vol_t = [B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dists[i]]._bdist.kwds['scale'] for i in range(2)]



        fig_cv['ax'][i][j].scatter(index, np.abs(B20.tk[tick].pd.lr['p']['all'].values), marker=markers[0], lw=0, s=ms, color=cols[7])
        for d in range(2):
            if Tmisc.get_sd_not_scale() and dists[d]=='t':
                nu = B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dists[d]]._bdist.kwds['df']
                factor = np.sqrt(nu/(nu-2))
            else:
                factor = 1
            for s in range(S):
                
                fig_cv['ax'][i][j].plot(index, factor*((-1)**s)*B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dists[d]]._bdist.kwds['scale'], lw=0.5, color=col_dists[d])



        fig_cv['ax'][i][j].set_xlim(xlim)
        fig_cv['ax'][i][j].set_ylim(ylim)


        


        xy_box = hwks_fns.annotate_model_Hawkes(ax=fig_cv['ax'][i][j], 
                                                #tick=tick, series="r_{{t}}", plot_tick=True, 
                                                str_tick=tick, plot_tick=True, 
                                                an_h='left', an_v='top', 
                                                an_x0=0.015, an_y0=0.025, flip_x0ry0t=True, an_mode=fstmp.get_an_mode())


        if i==0 and j==0:
            fig_cv['ax'][i][j].legend([fig_cv['ax'][i][j].get_children()[k] for k in [0,1*K,2*K]],
                                            np.concatenate([[f"$\\left|r_{{t}}\\right|$"], [f"$\\hat{{\\sigma}}_{{t}} | {cvol_fns.calc_cv_symb(vol='GARCH', o=0, dist=d)}$" for d in dists]]),
                                            loc="upper left",
                                            bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box], along_x=False, along_y=True)
                                            )
        
        fig_cv['ax'][i][j].tick_params(labelleft=(j==0), labelright=False, labelbottom=(i==I_max-1), labeltop=False)
        if i==I_max-1:
            fig_cv['ax'][i][j].set_xlabel(f"Date")
        if j==0:
            fig_cv['ax'][i][j].set_ylabel(f"$\\left|r_{{t}}\\right|$")



        fig_cv['fig'].set_size_inches(5.7,7)
        fig_cv['fig'].tight_layout()

fstmp.fig_export(fig_cv, height=7, formats=['pdf', 'png'], dpi=fstmp.get_dpi())

In [ ]:
vol = 'GARCH'

p = 1
o = 0
q = 1

resid_symb = f'\\tilde{{r}}'


dists = ["norm", "t"]


K = len(dists)
#dists_cols = [cols[3], cols[1], cols[5]]

mods_cols = [cols[9], cols[0]]

mods_select = np.arange(K)

mods_markers = ['v', '^']

ms = 9
marker = 'D'

xlim_abs = 4.25
ylim_abs = 13.5

xlim = xlim_abs*np.array([-1,1])
#ylim = ylim_abs*np.array([-1,1])
ylim = [-13.5, 7.5]

xs = np.linspace(-xlim_abs,xlim_abs,2)

CI = [0.95, 0.99]
N_CI  = len(CI)
cols_CI = plot_fns.calc_cols_CI(N_CI=N_CI)

for tick in B20.tk:
    mods = [B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist] for dist in dists]

    fig_norm_qq = plot_fns.rc_plot_setup(name=f'{tick}_tilde_r_qq_cv')


    data = copy.deepcopy(B20.tk[tick].pd.lr['p']['all'])
    

    period_t = B20.tk[tick].pd.get_periods_t_offset()['train']

    

    epsilon = [None for k in range(K)]
    tilde_r = [None for k in range(K)]
    factor = np.ones(K, dtype=int)


    
    T = np.size(data[period_t[0]:period_t[1]])
    fig_norm_qq_quantiles = (0.5+np.array(range(T)))/T
    fig_norm_qq_x = scipy.stats.norm().ppf(fig_norm_qq_quantiles)
    fig_norm_qq_x_ls = np.linspace(fig_norm_qq_x[0], fig_norm_qq_x[-1], 201)

    CI_y = np.linspace(ylim[0], ylim[1], 201)

    CI_x = scipy.stats.norm.ppf(np.array([[scipy.stats.norm().cdf(CI_y) + ((-1)**(1+k))*np.sqrt(np.log(2/((1-CI[j])/2)))/np.sqrt(2*T) for j in range(len(CI))] for k in range(2)]))

    
    #fig_norm_qq["ax"][0][0].scatter(fig_norm_qq_x, np.sort(data.values), color=cols[7], marker=marker, s=ms, lw=0)

    #fig_norm_qq["ax"][0][0].plot(fig_norm_qq_x[[0,-1]], normal_mean.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color='red', linewidth=1.5)
    #fig_norm_qq["ax"][0][0].plot(fig_norm_qq_x[[0,-1]], normal_median.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color="orange", linewidth=1.5) 
    if False:
        for i in range(N_CI):
            j = N_CI-(1+i)
            fig_norm_qq["ax"][0][0].fill_betweenx(y=CI_y, x1=CI_x[0][j], x2=CI_x[1][j], color=cols_CI[j])

    
    for m in mods_select:
        if Tmisc.get_sd_not_scale() and mods[m]._dist=='t':
            nu = mods[m]._bdist.kwds['df']
            factor[m] = np.sqrt(nu/(nu-2))

        epsilon[m] = np.sort(((data - mods[m]._bdist.kwds['loc'])/mods[m]._bdist.kwds['scale'])[period_t[0]:period_t[1]])
        tilde_r[m] = np.sort(scipy.stats.norm().ppf(mods[m].F_X[period_t[0]:period_t[1]]))

        

        fig_norm_qq["ax"][0][0].scatter(fig_norm_qq_x, tilde_r[m], color=mods_cols[m], marker=mods_markers[m], s=ms, lw=0)

        #fig_norm_qq["ax"][0][0].plot(fig_norm_qq_x_ls, B20.tk[tick]._stat_dist[dists[k]].ppf(scipy.stats.norm().cdf(fig_norm_qq_x_ls)), color=dists_cols[k], linewidth=1.5)

    
    #fig_norm_qq["ax"][0][0].plot(fig_norm_qq_x_ls, B20.tk[tick]._stat_dist['t_ML_f'].ppf(scipy.stats.norm().cdf(fig_norm_qq_x_ls)), color=cols[5], linewidth=1.5)
    fig_norm_qq["ax"][0][0].plot(xs, xs, color=cols[7], lw=2, zorder=-1)    

    #fig_norm_qq["ax"][0][0].set_ylabel(f"{tick} $r_t$")
    fig_norm_qq["ax"][0][0].set_ylabel(f"${resid_symb}_{{t}}$")
    fig_norm_qq["ax"][0][0].set_xlabel(f"Normal quantiles")


    fig_norm_qq["ax"][0][0].set_xlim(xlim)
    fig_norm_qq["ax"][0][0].set_ylim(ylim)
    #fig_norm_qq["ax"][0][0].set_ylim(0.25*np.array([-1,1]))

    if False:     
        u_q = np.array(range(2)) + np.array([1,-1])*B20.tk[tick].tt.tt["bi"]["mark"]._parameters.a_lambda.get_value()
        if True:
            u_y = B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value()
            u_x = (B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value() - normal_mean.mean())/normal_mean.std()
        else:
            u_y = normal_mean.ppf(u_q)
            u_x = scipy.stats.norm().ppf(u_q)
        for p in range(2):
            fig_norm_qq["ax"][0][0].plot(np.repeat(u_x[p], 2), np.array([u_y[p], ((-1)**(1+p))*5]), linewidth=0.5, linestyle="--", color="black")
            fig_norm_qq["ax"][0][0].plot(np.array([u_x[p], ((-1)**(1+p))*5]), np.repeat(u_y[p], 2), linewidth=0.5, linestyle="--", color="black")





    xy_box = hwks_fns.annotate_model_Hawkes(ax=fig_norm_qq["ax"][0][0], 
                                            #tick=tick, series="r_{{t}}", plot_tick=True, 
                                            str_tick=tick, plot_tick=True, 
                                            an_h='left', an_v='top', 
                                            an_x0=0.015, an_y0=0.03, flip_x0ry0t=True, an_mode=fstmp.get_an_mode())

    fig_norm_qq["ax"][0][0].legend([fig_norm_qq["ax"][0][0].get_children()[p] for p in np.arange(3)], 
                                   np.concatenate([[f"$\\hat{{{resid_symb}}}_{{t}} | {cvol_fns.calc_cv_symb(vol=vol, o=mods[m]._o, dist=mods[m]._dist)}$" for m in mods_select], [f"$\\mathcal{{N}}_{{\\left(0,1\\right)}}$"]]), loc="lower right",
                                   bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box], along_x=False, along_y=True)
                                   )


    fig_norm_qq["fig"].set_size_inches(5.7, 3)
    fig_norm_qq["fig"].tight_layout()

    fstmp.fig_export(fig_norm_qq, height=3, width=5.7, formats=['png', 'pdf'], dpi=fstmp.get_dpi())


In [ ]:
vol = 'GARCH'

p = 1
o = 0
q = 1

resid_symb = f'\\tilde{{r}}'


dists = ["norm", "t"]


K = len(dists)
#dists_cols = [cols[3], cols[1], cols[5]]

mods_cols = [cols[9], cols[0]]

mods_select = np.arange(K)

mods_markers = ['v', '^']

ms = 9
marker = 'D'

xlim_abs = 4.25
ylim_abs = 13.5

xlim = xlim_abs*np.array([-1,1])
#ylim = ylim_abs*np.array([-1,1])
ylim = [-13.5, 7.5]

xs = np.linspace(-xlim_abs,xlim_abs,2)

CI = [0.95, 0.99]
N_CI  = len(CI)
cols_CI = plot_fns.calc_cols_CI(N_CI=N_CI)


ticks = np.transpose([
                ["SPX", "DJI", "NKX"],
                ["DAX", "CAC", "HSI"],
            ])

I_max = np.shape(ticks)[0]
J_max = np.shape(ticks)[1]


fig_norm_qq = plot_fns.rc_plot_setup(name=f'TICK_tilde_r_qq_cv', nrows=I_max, ncols=J_max)

for i in range(I_max):
    for j in range(J_max):
        tick = ticks[i][j]

        mods = [B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist] for dist in dists]


        data = copy.deepcopy(B20.tk[tick].pd.lr['p']['all'])
        

        period_t = B20.tk[tick].pd.get_periods_t_offset()['train']

        

        epsilon = [None for k in range(K)]
        tilde_r = [None for k in range(K)]
        factor = np.ones(K, dtype=int)


        
        T = np.size(data[period_t[0]:period_t[1]])
        fig_norm_qq_quantiles = (0.5+np.array(range(T)))/T
        fig_norm_qq_x = scipy.stats.norm().ppf(fig_norm_qq_quantiles)
        fig_norm_qq_x_ls = np.linspace(fig_norm_qq_x[0], fig_norm_qq_x[-1], 201)

        CI_y = np.linspace(ylim[0], ylim[1], 201)

        CI_x = scipy.stats.norm.ppf(np.array([[scipy.stats.norm().cdf(CI_y) + ((-1)**(1+k))*np.sqrt(np.log(2/((1-CI[j])/2)))/np.sqrt(2*T) for j in range(len(CI))] for k in range(2)]))

        
        #fig_norm_qq["ax"][i][j].scatter(fig_norm_qq_x, np.sort(data.values), color=cols[7], marker=marker, s=ms, lw=0)

        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x[[0,-1]], normal_mean.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color='red', linewidth=1.5)
        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x[[0,-1]], normal_median.ppf(scipy.stats.norm().cdf(fig_norm_qq_x[[0,-1]])), color="orange", linewidth=1.5) 
        if False:
            for i in range(N_CI):
                j = N_CI-(1+i)
                fig_norm_qq["ax"][i][j].fill_betweenx(y=CI_y, x1=CI_x[0][j], x2=CI_x[1][j], color=cols_CI[j])

        
        for m in mods_select:
            if Tmisc.get_sd_not_scale() and mods[m]._dist=='t':
                nu = mods[m]._bdist.kwds['df']
                factor[m] = np.sqrt(nu/(nu-2))

            epsilon[m] = np.sort(((data - mods[m]._bdist.kwds['loc'])/mods[m]._bdist.kwds['scale'])[period_t[0]:period_t[1]])
            tilde_r[m] = np.sort(scipy.stats.norm().ppf(mods[m].F_X[period_t[0]:period_t[1]]))

            

            fig_norm_qq["ax"][i][j].scatter(fig_norm_qq_x, tilde_r[m], color=mods_cols[m], marker=mods_markers[m], s=ms, lw=0)

            #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x_ls, B20.tk[tick]._stat_dist[dists[k]].ppf(scipy.stats.norm().cdf(fig_norm_qq_x_ls)), color=dists_cols[k], linewidth=1.5)

        
        #fig_norm_qq["ax"][i][j].plot(fig_norm_qq_x_ls, B20.tk[tick]._stat_dist['t_ML_f'].ppf(scipy.stats.norm().cdf(fig_norm_qq_x_ls)), color=cols[5], linewidth=1.5)

        fig_norm_qq["ax"][i][j].plot(xs, xs, color=cols[7], lw=2, zorder=-1)

        #fig_norm_qq["ax"][i][j].set_ylabel(f"{tick} $r_t$")
        


        fig_norm_qq["ax"][i][j].set_xlim(xlim)
        fig_norm_qq["ax"][i][j].set_ylim(ylim)
        #fig_norm_qq["ax"][i][j].set_ylim(0.25*np.array([-1,1]))

        if False:     
            u_q = np.array(range(2)) + np.array([1,-1])*B20.tk[tick].tt.tt["bi"]["mark"]._parameters.a_lambda.get_value()
            if True:
                u_y = B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value()
                u_x = (B20.tk[tick].tt.tt["bi"]["mark"]._parameters.u.get_value() - normal_mean.mean())/normal_mean.std()
            else:
                u_y = normal_mean.ppf(u_q)
                u_x = scipy.stats.norm().ppf(u_q)
            for p in range(2):
                fig_norm_qq["ax"][i][j].plot(np.repeat(u_x[p], 2), np.array([u_y[p], ((-1)**(1+p))*5]), linewidth=0.5, linestyle="--", color="black")
                fig_norm_qq["ax"][i][j].plot(np.array([u_x[p], ((-1)**(1+p))*5]), np.repeat(u_y[p], 2), linewidth=0.5, linestyle="--", color="black")





        xy_box = hwks_fns.annotate_model_Hawkes(ax=fig_norm_qq["ax"][i][j], 
                                                #tick=tick, series="r_{{t}}", plot_tick=True, 
                                                str_tick=tick, plot_tick=True, 
                                                an_h='left', an_v='top', 
                                                an_x0=0.015, an_y0=0.03, flip_x0ry0t=True, an_mode=fstmp.get_an_mode())

        if i==0 and j==0:
            fig_norm_qq["ax"][i][j].legend([fig_norm_qq["ax"][i][j].get_children()[p] for p in np.arange(3)], 
                                np.concatenate([[f"$\\hat{{{resid_symb}}}_{{t}} | {cvol_fns.calc_cv_symb(vol=vol, o=mods[m]._o, dist=mods[m]._dist)}$" for m in mods_select], [f"$\\mathcal{{N}}_{{\\left(0,1\\right)}}$"]]), loc="lower right",
                                bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box], along_x=False, along_y=True)
                                )
        

        fig_norm_qq['ax'][i][j].tick_params(labelleft=(j==0), labelright=False, labelbottom=(i==I_max-1), labeltop=False)
        if i==I_max-1:
            fig_norm_qq["ax"][i][j].set_xlabel(f"Normal quantiles")
        if j==0:
            fig_norm_qq["ax"][i][j].set_ylabel(f"${resid_symb}_{{t}}$")

        
        


fig_norm_qq["fig"].set_size_inches(5.7, 7)
fig_norm_qq["fig"].tight_layout()

fstmp.fig_export(fig_norm_qq, height=7, width=5.7, formats=['png', 'pdf'], dpi=fstmp.get_dpi())



In [ ]:
def print_tab_r_stat(do_print=None, do_write=None, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, print_ftab=None, ftab_size=None,
                     phantom_sign=None, phantom_brac=None, sign_inside_brac=None, scif_string_format=None):

    if print_ftab is None:
        print_ftab = True

    C_str = fstmp.calc_C_str()


    tab_str = []
    ftab_str = []

    scif_string_spec = {
            #'exp_zeros': 1,
            'dp': 2,
            'phantom_sign': phantom_sign, 
            'phantom_brac': phantom_brac,
            'sign_inside_brac': sign_inside_brac, 
            'scif_string_format': scif_string_format,
        }

    ticks = [
            ['SPX', 'DAX', 'NKX'],
            ['DJI', 'CAC', 'HSI'],
        ]

    ticks = np.concatenate(np.transpose(ticks))


    tick_info = {tick: {info: None for info in ['Name', 'Exchange', 'City', 'Country']} for tick in ticks}

    for tick in ['SPX', 'DJI']:
        tick_info[tick]['Exchange'] = 'New York Stock Exchange'
        tick_info[tick]['City'] = 'New York'
        tick_info[tick]['Country'] = 'USA'




    tick_info['DAX']['Exchange'] = 'Frankfurt Stock Exchange'
    tick_info['DAX']['City'] = 'Frankfurt'
    tick_info['DAX']['Country'] = 'Germany'

    tick_info['CAC']['Exchange'] = 'Euronext Paris'
    tick_info['CAC']['City'] = 'Paris'
    tick_info['CAC']['Country'] = 'France'


    tick_info['NKX']['Exchange'] = 'Tokyo Stock Exchange'
    tick_info['NKX']['City'] = 'Tokyo'
    tick_info['NKX']['Country'] = 'Japan'

    tick_info['HSI']['Exchange'] = 'Hong Kong Stock Exchange'
    tick_info['HSI']['City'] = 'Hong Kong'
    tick_info['HSI']['Country'] = 'HKSAR'


    rows = ['head', 
                #'Exchange', 
                'City', 'Country', 'date_0', 'date_1', 'date_1+T', 'T', 'Xbar', 'sigma_X', 'Q_X', 'MAD_X']
    row_symbols =   {
                        'head': f"\\multicolumn{{1}}{{c}}{{}}",
                        'Exchange': f"Exchange",
                        'City': f"City",
                        'Country': f"Country",
                        'date_0': f"$\\mathrm{{Date}}_{{0}}$",
                        'date_1': f"$\\mathrm{{Date}}_{{1}}$",
                        'date_1+T': f"$\\mathrm{{Date}}_{{1+T}}$",
                        'T': f"$\\Delta{{T}}$",
                        
                    }

    if False:
        row_symbols['Xbar'] =  f"$\\bar{{\\rsPr}}$"
        row_symbols['sigma_X'] = f"$\\sigma_{{\\rsPr}}$"
        row_symbols['Q_X'] = f"$Q_{{0.5}}{{\\left(\\rsPr\\right)}}$"
        row_symbols['MAD_X'] = f"$\\mathrm{{MAD}}_{{\\rsPr}}$"
    else:
        row_symbols['Xbar'] =  f"$\\mathrm{{Mean}}{{\\left(\\rsPr_{{0:.T}}\\right)}}$"
        row_symbols['sigma_X'] = f"$\\mathrm{{SD}}{{\\left(\\rsPr_{{0:.T}}\\right)}}$"
        row_symbols['Q_X'] = f"$Q_{{0.5}}{{\\left(\\rsPr_{{0:.T}}\\right)}}$"
        row_symbols['MAD_X'] = f"$\\mathrm{{MAD}}{{\\left(\\rsPr_{{0:.T}}\\right)}}$"


    periods = ['train']

    t_str_w = {row: {period: '' for period in periods} for row in row_symbols}
    #t_str = {row: {period: {tick: '' for tick in ticks} for period in ['train', 'forecast']} for row in rows}
    t_str = {row: {period: ['' for tick in ticks] for period in periods} for row in row_symbols}


    left_r = 1


    for period in t_str['head']:
        #for tick in t_str['head'][period]:
            #t_str['head'][period][tick] = f"& \\multicolumn{{1}}{{c}}{{{tick}}} "
        for t in range(len(ticks)):
            tick = ticks[t]
            t_str['head'][period][t] = f"& \\multicolumn{{1}}{{l}}{{{tick}}} "

            if period=='train':
                data = B20.tk[tick].pd.lr['p']['train'].values
            else:
                data = B20.tk[tick].pd.lr['p']['forecast'][0][0].values

            T = data.size
            mean = np.mean(data)
            std = np.std(data) * np.sqrt(T/(T-1))

            median = np.quantile(data, 0.5)
            mad = np.quantile(np.abs(data-median), 0.5)



            if True:
                t_str['Exchange'][period][t] = f"& {tick_info[tick]['Exchange']} "
                t_str['City'][period][t] = f"& {tick_info[tick]['City']} "
                t_str['Country'][period][t] = f"& {tick_info[tick]['Country']} "
                t_str['date_0'][period][t] = f"& \\{C_str['l']}rsDatePriceAllStart{tick} "
                t_str['date_1'][period][t] = f"& \\{C_str['l']}rsDateAllStart{tick} "
                t_str['date_1+T'][period][t] = f"& \\{C_str['l']}rsDateAllEnd{tick} "
                t_str['T'][period][t] = f"& {misc_fns.capp_dlrs_tex_num(S=T, app_dlrs=True, app_tex_num=True)} "
                t_str['Xbar'][period][t] = f"& {misc_fns.scif_string(mean, **scif_string_spec)} "
                t_str['sigma_X'][period][t] = f"& {misc_fns.scif_string(std, **scif_string_spec)} "
                t_str['Q_X'][period][t] = f"& {misc_fns.scif_string(median, **scif_string_spec)} "
                t_str['MAD_X'][period][t] = f"& {misc_fns.scif_string(mad, **scif_string_spec)} "
            else:
                t_str['T'][period][t] = f"& ${T}$ "
                t_str['Xbar'][period][t] = f"& {misc_fns.scif_string(mean, dp=1, app_dlrs=True)} "
                t_str['sigma_X'][period][t] = f"& {misc_fns.scif_string(std, dp=1, app_dlrs=True)} "
                

            for row in rows:
                t_str_w[row][period] += t_str[row][period][t]



    C = len(t_str['head']['train'])
    #tab_str.append(f"\\begin{{tabular}}{{{c_str}}}")

    sample_string = {
                        "train": f"& \\multicolumn{{{C}}}{{c}}{{In-sample [{B20.tk[ticks[0]].pd._dates['train'][0].strftime('%Y-%m-%d')}..{B20.tk[ticks[0]].pd._dates['train'][-1].strftime('%Y-%m-%d')})}}",
                        #"forecast": f"& \\multicolumn{{{N_ticks}}}{{c}}{{Out-of-sample ({B20.tk[ticks[0]].pd._dates['train'][-1].strftime('%Y-%m-%d')} -- {B20.tk[ticks[0]].pd._dates['all'][-1].strftime('%Y-%m-%d')})}}",
                    }
    

    J = len(periods)

    C_J = J*(1+C) - 1
    

    tab_macro_str = f"TABrStat"
    tab_mfn_str = f"\\{C_str['l']}{tab_macro_str}"
            
    if print_ftab:
        c_str = misc_fns.str_concatenate(np.concatenate([np.repeat('r', left_r), np.repeat('l', C_J)]))
        ftab_str = plot_fns.calc_ftab_str(tab_mfn_str=tab_mfn_str, c_str=c_str, sizes=None, size=ftab_size, ftab_str=ftab_str)


    tab_str.append(f"\\newcommand{{{tab_mfn_str}}}{{")
    for period in t_str_w[rows[0]]:

        if False:
            tab_str.append(f"\\multicolumn{{1}}{{c}}{{}}")
            #tab_str.append(f"& \\multicolumn{{{N_ticks}}}{{c}}{{In-sample [\\rsDateTrainStart{{}}..\\rsDateTrainEnd{{}})}}")
            #tab_str.append(f"& \\multicolumn{{{N_ticks}}}{{c}}{{Out-of-sample [\\rsDateTrainEnd{{}}..\\rsDateAllEnd{{}})}}")
            tab_str.append(sample_string[period])


        #tab_str.append(f"\\\\") 
        #tab_str.append(f"\\cmidrule{{{2+0*N_ticks}-{2+1*N_ticks-1}}} \cmidrule{{{2+1*N_ticks}-{2+2*N_ticks-1}}}")
        #tab_str.append(f"\\cmidrule{{{2+0*N_ticks}-{2+1*N_ticks-1}}}")

        for row in rows:
            tab_str.append(row_symbols[row])
            tab_str.append(t_str_w[row][period])
            tab_str.append("\\\\")
            if row=='head':
                tab_str.append('\\hline')
            elif row=='date_1+T':
                tab_str.append('\\hdashline')
            #elif row==rows[-1] and period!="forecast":
                #tab_str.append('\\hline')

        #tab_str.append(f"\\end{{tabular}}")
    tab_str.append(f"}}")
    tab_str.append(f"")

    if file_nm is None:
        file_nm = "r_stat"

    fstmp.print_str_pw(print_str=tab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"tab_{file_nm}", file_ext=file_ext)
    if print_ftab:
        fstmp.print_str_pw(print_str=ftab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"ftab_{file_nm}", file_ext=file_ext)

In [ ]:
phantom_sign=True
phantom_brac=False
sign_inside_brac=None
scif_string_format='E'
ftab_size=None

print_tab_r_stat(do_print=True, do_write=True, write_mode=None, file_nm=None, ftab_size=ftab_size,
                 phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

In [ ]:
tick='SPX'

param_descale = ""

x_str = "x"
if Tmisc.get_sd_not_scale():
    scale_str = "\\varsigma"
else:
    scale_str = "\\sigma"

if param_descale=="ls":
    arg_str = f"{x_str}"
    param_str = f"0,1,\\nu"
elif param_descale=="l":
    arg_str = f"{x_str}/{scale_str}"
    param_str = f"0,{scale_str},\\nu"
else:
    #arg_str = f"\\left(x-\mu\\right)/{scale_str}"
    #arg_str = f"\\left\{{x-\mu\\right\}}/{scale_str}"
    arg_str = f"\\left[{x_str}-\mu\\right]/{scale_str}"
    #arg_str = f"\\frac{{x-\mu}}{{{scale_str}}}"
    param_str = f"\\mu,{scale_str},\\nu"


for tick in B20.tk:
    fig_students = plot_fns.rc_plot_setup(name=f"{tick}_students_t", nrows=2)

    funcs = ['cdf', 'pdf']


    nu_s = np.array([1,2,4,8])
    xs_max = 4
    xs = xs_max*np.linspace(-1,1,801)

    nu_col = cols[7]

    for i in range(2):
        func = funcs[i]

        for nu in nu_s:
            fig_students['ax'][i][0].plot(xs, scipy.stats.t(loc=0, scale=1, df=nu).__getattribute__(func)(xs), lw=1, color=nu_col)
            #fig_students['ax'][1][0].plot(xs, scipy.stats.t(loc=0, scale=1, df=nu).cdf(xs), lw=1, color=nu_col)

        if False:
            fig_students['ax'][i][0].plot(xs, scipy.stats.t(loc=0, scale=1, df=np.infty).__getattribute__(func)(xs), lw=2, color=cols[9])
        else:
            fig_students['ax'][i][0].plot(xs, scipy.stats.norm(loc=0, scale=1).__getattribute__(func)(xs), lw=2, color=cols[9])
        fig_students['ax'][i][0].plot(xs, scipy.stats.t(loc=0, scale=1, df=B20.tk[tick]._stat_dist['t_ML'].args[0]).__getattribute__(func)(xs), lw=2, color=cols[0])


        #fig_students['ax'][i][0].axhline(0, lw=0.5, color='black')
        

        fig_students['ax'][i][0].set_xlim(xs_max*np.array([-1,1]))


    fig_students['ax'][0][0].set_ylim([0,1])
    fig_students['ax'][1][0].set_ylim([0, 1.04*scipy.stats.norm().pdf(0)])

    fig_students['ax'][-1][0].set_xlabel(f"${arg_str}$")

    fig_students['ax'][0][0].set_ylabel(f"$F^{{\mathcal{{S}}_{{\\left({param_str}\\right)}}}}{{\\left({arg_str}\\right)}}$")
    fig_students['ax'][1][0].set_ylabel(f"$f^{{\mathcal{{S}}_{{\\left({param_str}\\right)}}}}{{\\left({arg_str}\\right)}}$")

    fig_students['ax'][0][0].legend([fig_students['ax'][0][0].get_children()[k] for k in np.concatenate([[0], nu_s.size+np.arange(2)])],
                                    #[f"$\\nu \in \\left\\{{{misc_fns.str_concatenate(nu_s, fill_str=',', fill_str_void=-1)}\\right\\}}$",
                                    [f"$\\nu \in {misc_fns.vector_string(nu_s, dp=0, scif_pow10=0, app_dlrs=False, brac_type='{}', add_lr=True)}$",
                                    f"$\\nu = \\infty | \\mathcal{{N}}$",
                                    #f"$\\nu = {B20.tk[tick]._stat_dist['t_ML'].args[0]:.2f} | \\mathcal{{S}}_{{\mathrm{{ML}}}}^{{\\mathrm{{{tick}}}}}$",
                                    f"$\\nu = {B20.tk[tick]._stat_dist['t_ML'].args[0]:.2f} | \\hat{{\\mathcal{{S}}}}_{{W}}^{{\\mathrm{{{tick}}}}}$",
                                    ],
                                    loc='lower right')
    


    fig_students['fig'].set_size_inches(5.7,7)
    fig_students['fig'].tight_layout()

    fstmp.fig_export(fig_students, height=7)

In [ ]:
tick='SPX'
mode='bi'
mark='mark'

u = 0.05 * np.array([-1,1])


for tick in B20.tk:
    t_event_bool = np.array([((-1)**k)*B20.tk[tick].tt.tt[mode][mark]._df.values[:B20.tk[tick].tt.tt[mode][mark]._train_t[-1],0] < ((-1)**k)*u[k] for k in range(2)])
    N = np.sum(t_event_bool, axis=1)
    a_lambda = N/B20.tk[tick].tt.tt[mode][mark]._train_t[-1]

    for mode in B20.tk[tick].tt.tt:
        for mark in B20.tk[tick].tt.tt[mode]:
            B20.tk[tick].tt.tt[mode][mark]._parameters.u.set_value(u)
            B20.tk[tick].tt.tt[mode][mark]._parameters.a_u.set_value(a_lambda)
            B20.tk[tick].tt.tt[mode][mark]._parameters.a_lambda.set_value(a_lambda)
            B20.tk[tick].tt.tt[mode][mark].detect_events()


tick='SPX'
mode='bi'
mark='mark'

print(B20.tk[tick].tt.tt[mode][mark]._parameters.u.get_value())
print(B20.tk[tick].tt.tt[mode][mark]._parameters.a_u.get_value())
print(B20.tk[tick].tt.tt[mode][mark]._parameters.a_lambda.get_value())

In [ ]:
param_descale = "Pls"
cmap = "viridis_r"

x_str = "x"
scale_str = "\\sigma"
#df_str = '\\nu'
df_str = 'D'

if param_descale=="Pls":
    arg_str = f"{x_str}"
    param_str = f"{df_str}"
elif param_descale=="ls":
    arg_str = f"{x_str}"
    param_str = f"0,1,{df_str}"
elif param_descale=="l":
    arg_str = f"{x_str}/{scale_str}"
    param_str = f"0,{scale_str},{df_str}"
else:
    #arg_str = f"\\left(x-\mu\\right)/{scale_str}"
    #arg_str = f"\\left\{{x-\mu\\right\}}/{scale_str}"
    arg_str = f"\\left[{x_str}-\mu\\right]/{scale_str}"
    #arg_str = f"\\frac{{x-\mu}}{{{scale_str}}}"
    param_str = f"\\mu,{scale_str},{df_str}"




fig_chi_squared = plot_fns.rc_plot_setup(name=f"fig_chi-squared_{cmap}", nrows=2)

funcs = ['cdf', 'pdf']


nu_s = np.array([1,2,3,4,6,9])
xs_max = 8
xs = xs_max*np.linspace(0,1,801)

nu_col = cols[7]

for i in range(2):
    func = funcs[i]

    if cmap=="":
        fig_chi_squared['ax'][i][0].plot(xs, scipy.stats.chi2(loc=0, scale=1, df=1).__getattribute__(func)(xs), lw=2, color=cols[9])
        for nu in nu_s:
            fig_chi_squared['ax'][i][0].plot(xs, scipy.stats.chi2(loc=0, scale=1, df=nu).__getattribute__(func)(xs), lw=1, color=nu_col)
            #fig_chi_squared['ax'][1][0].plot(xs, scipy.stats.t(loc=0, scale=1, df=nu).cdf(xs), lw=1, color=nu_col)
    else:
        nu_s = np.sort(nu_s)
        N_nu = np.size(nu_s)
        nu_s_F = (nu_s - nu_s[0])/np.diff(nu_s[[0,N_nu-1]])[0]
        cm = matplotlib.cm.get_cmap(cmap)
        for n in range(N_nu):
            fig_chi_squared['ax'][i][0].plot(xs, scipy.stats.chi2(loc=0, scale=1, df=nu_s[n]).__getattribute__(func)(xs), lw=2, color=cm(nu_s_F[n]))


        

    

    fig_chi_squared['ax'][i][0].set_xlim(xs_max*np.array([0,1]))


fig_chi_squared['ax'][0][0].set_ylim([0,1])
fig_chi_squared['ax'][1][0].set_ylim([0, 0.5])

fig_chi_squared['ax'][-1][0].set_xlabel(f"${arg_str}$")

fig_chi_squared['ax'][0][0].set_ylabel(f"$F^{{\\chi^{{2}}_{{\\left({param_str}\\right)}}}}{{\\left({arg_str}\\right)}}$")
fig_chi_squared['ax'][1][0].set_ylabel(f"$f^{{\\chi^{{2}}_{{\\left({param_str}\\right)}}}}{{\\left({arg_str}\\right)}}$")

if cmap=="":
    fig_chi_squared['ax'][0][0].legend([fig_chi_squared['ax'][0][0].get_children()[k] for k in np.concatenate([[0], [1]])],
                                    #[f"$\\nu \in \\left\\{{{misc_fns.str_concatenate(nu_s, fill_str=',', fill_str_void=-1)}\\right\\}}$",
                                    [   
                                        f"$\\nu = 1$",
                                        f"$\\nu \in {misc_fns.vector_string(nu_s, dp=0, scif_pow10=0, app_dlrs=False, brac_type='{}', add_lr=True)}$",
                                    ],
                                    loc='lower right')
else:
     fig_chi_squared['ax'][0][0].legend([fig_chi_squared['ax'][0][0].get_children()[k] for k in range(N_nu)],
                                    #[f"$\\nu \in \\left\\{{{misc_fns.str_concatenate(nu_s, fill_str=',', fill_str_void=-1)}\\right\\}}$",
                                    [f"$\\nu = {nu}$" for nu in nu_s],
                                    loc='lower right')




fig_chi_squared['fig'].set_size_inches(5.7,7)
fig_chi_squared['fig'].tight_layout()

fstmp.fig_export(fig_chi_squared, height=7)

In [ ]:
param_descale = ""

x_str = "M"
scale_str = "\\varsigma"
tf_u_key = 1


if param_descale=="s":
    arg_str = f"{x_str}"
    param_str = misc_fns.capp_brac(f"1,\\xi", brac_type="()", add_lr=True)
else:
    arg_str = f"{x_str}/{scale_str}"
    param_str = misc_fns.capp_brac(f"{scale_str},\\xi", brac_type="()", add_lr=True)



for tick in B20.tk:

    fig_pareto = plot_fns.rc_plot_setup(name=f"{tick}_pareto", nrows=2)

    funcs = ['cdf', 'pdf']


    xi_s = np.array([-1,-0.75,-0.5,-0.25,0.5,1,2])


    #xi_s_str = [f"{xi:+.2f}" for xi in xi_s]
    xs_max = 5
    xs = xs_max*np.linspace(0,1,801)

    arrows = [f"\\leftarrow", f"\\rightarrow"]

    nu_col = cols[7]

    tf_u_key = 1
    a_u = B20.tk[tick]._tail_fit['a_u']['lambda']['a_u'][0,tf_u_key]


    for i in range(2):
        func = funcs[i]

        for xi in xi_s:
            fig_pareto['ax'][i][0].plot(xs, scipy.stats.genpareto(scale=1, c=xi).__getattribute__(func)(xs), lw=1, color=nu_col)
            #fig_students['ax'][1][0].plot(xs, scipy.stats.t(loc=0, scale=1, df=nu).cdf(xs), lw=1, color=nu_col)

        #fig_pareto['ax'][i][0].plot(xs, scipy.stats.t(loc=0, scale=1, df=B20.tk[tick]._stat_dist['t_ML'].args[0]).__getattribute__(func)(xs), lw=1.5, color=cols[0])
        fig_pareto['ax'][i][0].plot(xs, scipy.stats.genpareto(scale=1, c=0).__getattribute__(func)(xs), lw=2, color=cols[9])
        for k in range(2):
            #fig_pareto['ax'][i][0].plot(xs, scipy.stats.genpareto(scale=1, c=B20.tk[tick].tt.tt[mode][mark]._tail_gp_fit[k]['est'][0]).__getattribute__(func)(xs), lw=2, color=cols_tail[0][k])
            fig_pareto['ax'][i][0].plot(xs, scipy.stats.genpareto(scale=1, c=B20.tk[tick]._tail_fit['a_u']['gp_fit']['est'][k,tf_u_key,0]).__getattribute__(func)(xs), lw=2, color=cols_tail[0][k])

        fig_pareto['ax'][i][0].set_xlim(xs_max*np.array([0,1]))
        #fig_pareto['ax'][i][0].set_ylim(plot_fns.buffer(b=1.04, x=np.array([0,1])))
        fig_pareto['ax'][i][0].set_ylim(plot_fns.buffer(b=1, x=np.array([0,1])))


    fig_pareto['ax'][-1][0].set_xlabel(f"${arg_str}$")

    fig_pareto['ax'][0][0].set_ylabel(f"$F^{{\mathcal{{P}}_{{{param_str}}}}}{{{misc_fns.capp_brac(arg_str, add_lr=True)}}}$")
    fig_pareto['ax'][1][0].set_ylabel(f"$f^{{\mathcal{{P}}_{{{param_str}}}}}{{{misc_fns.capp_brac(arg_str, add_lr=True)}}}$")


    fig_pareto['ax'][0][0].legend([fig_pareto['ax'][0][0].get_children()[k] for k in np.concatenate([[0], xi_s.size+np.arange(3)])],
                                        np.concatenate([
                                        #[f"$\\xi \in \\left\\{{{misc_fns.str_concatenate(xi_s_str, fill_str=',', fill_str_void=-1)}\\right\\}}$",
                                        [f"$\\xi \in {misc_fns.vector_string(xi_s, dp=2, scif_pow10=0, app_dlrs=False, brac_type='{}', add_lr=True, val_plus=True)}$",
                                        f"$\\xi = {0} | \\mathcal{{E}}$"],
                                        [
                                        #f"$\\xi = {B20.tk[tick]._tail_fit['a_u']['gp_fit']['est'][k,tf_u_key,0]:+.2f} | \\mathcal{{P}}_{{\mathrm{{ML}}}}^{{{arrows[k]}, \\mathrm{{{tick}}}}}{{\\left(a_u = {a_u:.3f}\\right)}}$"
                                        #f"$\\xi = {B20.tk[tick]._tail_fit['a_u']['gp_fit']['est'][k,tf_u_key,0]:+.2f} | \\mathcal{{P}}_{{\mathrm{{ML}}, a_u = {a_u:.3f}}}^{{{arrows[k]}, \\mathrm{{{tick}}}}}$"
                                        f"$\\xi = {B20.tk[tick]._tail_fit['a_u']['gp_fit']['est'][k,tf_u_key,0]:+.2f} | \\hat{{\\mathcal{{P}}}}_{{M,U{{(a_u = {a_u:.3f})}}}}^{{{arrows[k]}, \\mathrm{{{tick}}}}}$"
                                        for k in range(2)]
                                        ]),
                                        loc='lower right')


    fig_pareto['fig'].set_size_inches(5.7,7)
    fig_pareto['fig'].tight_layout()

    fstmp.fig_export(fig_pareto, height=7)

In [ ]:
B20.tk[tick]._tail_fit['a_u']['gp_fit']['est']

In [ ]:
for tick in B20.tk:
    B20.tk[tick]._tail_fit['u_fine'] = dist_fns.calc_tail_fit(B20.tk[tick].pd.lr["p"]["all"].values, u=np.outer([-1,1], np.linspace(0.0025, 0.16, 64))) 
    B20.tk[tick]._tail_fit['a_u_fine'] = dist_fns.calc_tail_fit(B20.tk[tick].pd.lr["p"]["all"].values, u_dir=[-1,1], a_u=np.array([np.linspace(0.0025, 0.25, 100)])) 

In [ ]:
def plot_U_u_key(ticks=None, u_key=None, incl_N_ax2=None, split_arrow_legend=None, tick_top=None, xlim=None, xticks=None, height=None, width=None): 

    if u_key is None:
        u_key = 'u'

    if width is None:
        width = 5.7

    if height is None:
        height = 7

    
    an_params = plot_fns.calc_an_params(an_mode='thesis')

    if incl_N_ax2 is None:
        incl_N_ax2 = False

    if split_arrow_legend is None:
        split_arrow_legend = True

    if tick_top is None:
        tick_top = True

    if ticks is None:
        ticks = [B20._ticks]
    ticks_flat = misc_fns.make_iterable_array(ticks)
    T = {tick: B20.tk[tick]._T['all'] for tick in ticks_flat}

    arrows = [f'\\leftarrow', f'\\rightarrow']
    if xlim is None:
        if u_key=='u':
            xlim=[0,0.08]
        else:
            #xlim=[0,0.25]
            xlim=[0.0025,0.25]


    if u_key=='u':
        I_max = 5
        logscale_i = np.concatenate([[0]])
        logscale_j = False
        arrow_v = 'upper'
        arrow_h = 'left'
        mod_v = 'bottom'
        
    else:
        I_max = 5
        #logscale_i = np.concatenate([[0], I_max-np.array([4,2])])
        logscale_i = np.concatenate([[]])
        #arrow_v = 'lower'
        arrow_v = 'upper'
        arrow_h = 'right'
        mod_v = 'top'
        logscale_j = True
        

    
        

    plot_bool = {tick: (np.abs(B20.tk[tick]._tail_fit[f'{u_key}_fine']['lambda'][f'{u_key}'][0,:])>=xlim[0])*(np.abs(B20.tk[tick]._tail_fit[f'{u_key}_fine']['lambda'][f'{u_key}'][0,:])<=xlim[1]) for tick in ticks_flat}

    for ticks_i in ticks:

        J_max = len(ticks_i)

        fig_U_u_key = plot_fns.rc_plot_setup(name=f"TICK_U_{u_key}_r_{misc_fns.str_concatenate(ticks_i)}", nrows=I_max, ncols=J_max)


        

        if u_key=='u' and incl_N_ax2:
            fig_U_u_key['ax2'] = [[fig_U_u_key['ax'][i][j].twinx() for j in range(J_max)] for i in range(1)] #range(I_max)]
            for i in range(len(fig_U_u_key['ax2'])):
                for j in range(len(fig_U_u_key['ax2'][i])):
                    fig_U_u_key['ax2'][i][j].tick_params(left=False, right=False, top=False, bottom=False,
                                                    labelleft=False, labelright=False, labeltop=False, labelbottom=False)

            for j in range(len(fig_U_u_key['ax2'][i])):
                fig_U_u_key['ax'][0][j].tick_params(right=False,
                                                        labelleft=False, labelright=False, labeltop=False, labelbottom=False)
                fig_U_u_key['ax2'][0][j].tick_params(right=True)
                fig_U_u_key['ax2'][0][j].yaxis.set_label_position("right")
            fig_U_u_key['ax2'][0][-1].tick_params(labelright=True)
            fig_U_u_key['ax'][0][0].tick_params(labelleft=True)
        

        for j in range(len(ticks_i)):
            tick = ticks_i[j]
            for k in range(2):

                u_key_x = np.abs(B20.tk[tick]._tail_fit[f'{u_key}_fine']['lambda'][f'{u_key}'][k,plot_bool[tick]])
                
                if u_key=='u':
                    fig_U_u_key['ax'][0][j].plot(u_key_x, B20.tk[tick]._tail_fit[f'{u_key}_fine']['lambda']['N'][k,plot_bool[tick]]/T[tick], lw=1, color=cols_tail[0][k])
                    #fig_U_u_key['ax'][1][j].plot(u_key_x, B20.tk[tick]._tail_fit[f'{u_key}_fine']['M_stat']['mean'][k,plot_bool[tick]], lw=1, color=cols_tail[0][k])
                    if False:
                        fig_U_u_key['ax'][1][j].fill_between(x=u, y1=B20.tk[tick]._tail_fit[f'{u_key}_fine']['M_stat']['mean'][k,plot_bool[tick]]-B20.tk[tick]._tail_fit[f'{u_key}_fine']['M_stat']['std'][k,plot_bool[tick]], 
                                                                    y2=B20.tk[tick]._tail_fit[f'{u_key}_fine']['M_stat']['mean'][k,plot_bool[tick]]+B20.tk[tick]._tail_fit[f'{u_key}_fine']['M_stat']['std'][k,plot_bool[tick]], 
                                                                    color=cols_tail[0][k], alpha=0.2)
                else:
                    fig_U_u_key['ax'][0][j].plot(u_key_x, np.abs(B20.tk[tick]._tail_fit[f'{u_key}_fine']['lambda'][f'u'][k,plot_bool[tick]]), lw=1, color=cols_tail[0][k])
                
                fig_U_u_key['ax'][I_max-4][j].plot(u_key_x, B20.tk[tick]._tail_fit[f'{u_key}_fine']['M_stat']['mean'][k,plot_bool[tick]], lw=1, color=cols_tail[0][k])
                    
                for i in range(2):
                    fig_U_u_key['ax'][I_max-3+i][j].plot(u_key_x, B20.tk[tick]._tail_fit[f'{u_key}_fine']['gp_fit']['est'][k,plot_bool[tick],2*i], lw=1, color=cols_tail[0][k])
                    fig_U_u_key['ax'][I_max-3+i][j].fill_between(x=u_key_x, y1=B20.tk[tick]._tail_fit[f'{u_key}_fine']['gp_fit']['est'][k,plot_bool[tick],2*i]-B20.tk[tick]._tail_fit[f'{u_key}_fine']['gp_fit']['se'][k,plot_bool[tick],2*i], 
                                                            y2=B20.tk[tick]._tail_fit[f'{u_key}_fine']['gp_fit']['est'][k,plot_bool[tick],2*i]+B20.tk[tick]._tail_fit[f'{u_key}_fine']['gp_fit']['se'][k,plot_bool[tick],2*i], 
                                                            color=cols_tail[0][k], alpha=0.2)
                    

                fig_U_u_key['ax'][I_max-1][j].plot(u_key_x, -2*B20.tk[tick]._tail_fit[f'{u_key}_fine']['gp_fit']['eval']['log_L'][k,plot_bool[tick]]/B20.tk[tick]._tail_fit[f'{u_key}_fine']['lambda']['N'][k,plot_bool[tick]], lw=1, color=cols_tail[0][k])


            for i in logscale_i:
                fig_U_u_key['ax'][i][j].set_yscale('log')
                if u_key=='u' and incl_N_ax2 and i==0:
                    fig_U_u_key['ax2'][i][j].set_yscale('log')
            if logscale_j:
                for i in range(I_max):
                    fig_U_u_key['ax'][i][j].set_xscale('log')
                    if u_key=='u' and incl_N_ax2 and i==0:
                        fig_U_u_key['ax2'][i][j].set_xscale('log')

            fig_U_u_key['ax'][0][j].tick_params(labeltop=False)
            if u_key=='u' and incl_N_ax2:
                fig_U_u_key['ax2'][0][j].tick_params(labeltop=False, top=bool(1-tick_top))

        

            for i in range(I_max):
                if u_key=='u':
                    fig_U_u_key['ax'][i][j].set_xlim(np.flip(xlim))
                else:
                    fig_U_u_key['ax'][i][j].set_xlim(xlim)
                if xticks is not None:
                    fig_U_u_key['ax'][i][j].set_xticks(xticks)

            if u_key=='u':
                fig_U_u_key['ax'][-1][j].set_xlabel(f"$|u^{{\\leftrightharpoons}}|$")
            else:
                fig_U_u_key['ax'][-1][j].set_xlabel(f"$a_{{u}}$")

            if tick_top:
                fig_U_u_key['ax'][0][j].set_xlabel(f"$r_{{t}}$ [{tick.upper()}]")
                fig_U_u_key['ax'][0][j].xaxis.set_label_position("top")
                fig_U_u_key['ax'][0][j].tick_params(top=False, labeltop=False)


        ax_i_j_ylims = np.array([[fig_U_u_key['ax'][i][j].get_ylim() for j in range(J_max)] for i in range(I_max)])
        ax_i_ylim = np.transpose([np.min(ax_i_j_ylims[:,:,0], axis=1), np.max(ax_i_j_ylims[:,:,1], axis=1)])

        for i in range(I_max):
            fig_U_u_key['ax'][i][-1].tick_params(labelright=False)
            for j in range(J_max):
                fig_U_u_key['ax'][i][j].set_ylim(ax_i_ylim[i,:])
                if u_key=='u' and incl_N_ax2 and i==0:
                    fig_U_u_key['ax2'][i][j].set_ylim(ax_i_ylim[i,:]*T[ticks_i[j]])

        if u_key=='u':
            if incl_N_ax2:
                fig_U_u_key['ax2'][0][-1].tick_params(labelright=True)
                fig_U_u_key['ax2'][0][-1].set_ylabel(f"$\\Delta{{{GR.pltsbv('N')}}}$")
            fig_U_u_key['ax'][0][0].set_ylabel(f"$\\hat{{{GR.pltsbv('a')}}}_{{\\pi}}$")
            
        else:
            fig_U_u_key['ax'][0][0].set_ylabel(f"$\\{plot_fns.calc_tex_vect_fn()}{{|u|}}$")


        log_L_symb = '\\ell'
        fig_U_u_key['ax'][I_max-4][0].set_ylabel(f"$\\{plot_fns.calc_tex_vect_fn()}{{\\left<M\\right>}}$")
        fig_U_u_key['ax'][I_max-3][0].set_ylabel(f"${misc_fns.capp_tex_fn(GR.pltsbv('xi'), 'hat')}$")
        fig_U_u_key['ax'][I_max-2][0].set_ylabel(f"${misc_fns.capp_tex_fn(GR.pltsbv('varsigma'), 'hat')}$")
        #fig_U_u_key['ax'][I_max-1][0].set_ylabel(f'$\\ell^{{\\leftrightharpoons}}/N_{{T}}^{{\\leftrightharpoons}}$')
        fig_U_u_key['ax'][I_max-1][0].set_ylabel(f"$-2\\hat{{\\{plot_fns.calc_tex_vect_fn()}{{\\ell}}}}_{{M}}/\\Delta{{{GR.pltsbv('N')}}}$")
        #fig_U_u_key['ax'][I_max-1][0].set_ylabel(f"$-2{misc_fns.capp_tex_fn(plot_fns.capp_tex_vect(log_L_symb), 'hat')}_{{M}}/{GR.pltsbv('N')}_{{T}}$")



        xy_box = [hwks_fns.annotate_model_Hawkes(ax=fig_U_u_key['ax'][0][j], 
                                            tick=tick, series=f"r_{{t}}", plot_tick=bool(1-tick_top), 
                                            str_u=f"$\\Delta{{T}}={plot_fns.capp_tex_num(T[ticks_i[j]])}$", plot_u=True,
                                            an_h='right', an_v=mod_v, flip_x0ry0t=True,
                                            an_yi=0.13, an_y0=0.04, an_mode=fstmp.get_an_mode()) for j in range(len(fig_U_u_key['ax'][0]))]


        if split_arrow_legend:
            for k in range(2):
                fig_U_u_key['ax'][0][k].legend([fig_U_u_key['ax'][0][0].get_children()[k]],
                                                #[f"$N_{{T}}^{{{arrow}}}$" for arrow in arrows],
                                                [[f"${arrow}$" for arrow in arrows][k]],
                                                loc=f"{arrow_v} {arrow_h}",
                                                bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box[0]], along_x=False, along_y=True),
                                                fontsize=an_params['an_fs'])

        else:
            fig_U_u_key['ax'][0][0].legend([fig_U_u_key['ax'][0][0].get_children()[k] for k in range(2)],
                                                #[f"$N_{{T}}^{{{arrow}}}$" for arrow in arrows],
                                                [f"${arrow}$" for arrow in arrows],
                                                loc=f"{arrow_v} {arrow_h}",
                                                bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box[0]], along_x=False, along_y=True),
                                                fontsize=an_params['an_fs'])
        


        

        


        fig_U_u_key['fig'].set_size_inches(width,height)
        fig_U_u_key['fig'].tight_layout()

        fstmp.fig_export(fig_U_u_key, height=height, width=width)

    #return fig_U_u_key

In [ ]:
plot_U_u_key(ticks=np.transpose([B20._ticks]), u_key='u', incl_N_ax2=True, split_arrow_legend=False, tick_top=False, width=5.7, height=7)

In [ ]:
plot_U_u_key(ticks=np.transpose([B20._ticks]), u_key='a_u', incl_N_ax2=True, split_arrow_legend=False, tick_top=False, width=5.7, height=7)

In [ ]:
plot_U_u_key(ticks=[B20._ticks], u_key='u', incl_N_ax2=False, xticks=np.linspace(0.01, 0.07, 4), width=8.5, height=4.5)

In [ ]:
plot_U_u_key(ticks=[B20._ticks], u_key='a_u', incl_N_ax2=False, xticks=None, width=8.5, height=4.5)

In [ ]:
tf_mode = 'u'
tf_u_key = 3
min_axlabels = True

periods = ['all', 'zoom']
vals = ['N_t']
I_s = [[i] for i in range(2)]
Nlim_rf_upper_s = [10,5]
Nlim_rb_upper = 0.01

height = {1: 2.5, 2: 5, 3: 7}

KS_arrows = [f"\\leftarrow", f"\\rightarrow", f"\\leftrightarrow"]
ineqs = [f"<", f">"]

decades = 10*np.arange(193, 203)
decades_label = [f"{d}" for d in decades]

I_max = len(periods)
J_max = len(I_s)
p_KS = {tick: [[None for j in range(J_max)] for i in range(I_max)] for tick in B20.tk}

for tick in B20.tk:

   decades_label = [f"{d}" for d in decades]
   if np.isin(tick, ['SPX', 'DJI']):
        for k in range(decades.size//2):
            decades_label[2*k] = f""

   overwrite_t_ps = {'zoom': ['2008', '2024']}
   tf_args = {'tf_mode': tf_mode}
   if tf_u_key is not None:
        tf_args['tf_u_key'] = tf_u_key
        tf_args['a_u'] = B20.tk[tick]._tail_fit[tf_mode]['lambda']['a_u'][:,tf_u_key]
        tf_args['a_lambda'] = B20.tk[tick]._tail_fit[tf_mode]['lambda']['a_lambda'][:,tf_u_key]
        tf_args['u'] = B20.tk[tick]._tail_fit[tf_mode]['lambda']['u'][:,tf_u_key]

   if tf_mode=='u':
      u_name = tf_args['u']
   else:
      u_name = tf_args['a_u']

   name = f"{tick}_KS_{misc_fns.str_concatenate(vals, fill_str='-', fill_str_void=-1)}_{misc_fns.str_concatenate(periods, fill_str='-', fill_str_void=-1)}_U_{tf_mode}={str(np.abs(u_name[0])).replace('.','')}"

   fig_KS = plot_fns.rc_plot_setup(name=name, nrows=I_max, ncols=J_max)

   fig_KS['ax2'] = [[fig_KS['ax'][i][j].twiny() for j in range(len(fig_KS['ax'][i]))] for i in range(len(fig_KS['ax']))]

   for i in range(I_max):
         for j in range(J_max):

            fig_KS['ax2'][i][j].tick_params(left=False, right=False, top=False, bottom=False,
                                             labelleft=False, labelright=False, labeltop=False, labelbottom=False)


   for i in range(I_max):
         for j in range(J_max):
            p_KS[tick][i][j] = lr_fns.KS_plot(self=B20, ax=fig_KS['ax'][i][j], ax2=fig_KS['ax2'][i][j], tick=tick, period=periods[i], val_key=vals[0], t_bracket=False, thesis=fstmp.get_thesis(),
                                             **tf_args,
                                             I_s=I_s[j],
                                             plot_main_legend=i==0, plot_ci_legend=(i==0)*(j==0),
                                             overwrite_t_ps=overwrite_t_ps,
                                             Nlim_rf_upper=Nlim_rf_upper_s[i], Nlim_rb_upper=Nlim_rb_upper, 
                                             #an_y0=0.15, 
                                             #an_y0_t=0.21, 
                                             #u_scif_pow10=0,
                                             ci_legend_loc='lower right', ci_legend_ncols=1,
                                             )
            if True:
               if i==0:
                  xylim = {f"{a}lim": fig_KS['ax'][i][j].__getattribute__(f'get_{a}lim')() for a in ['x', 'y']} 

                  fig_KS["ax"][i][j].set_xticks([pd.Timestamp(f"{d}") for d in decades])
                  fig_KS["ax"][i][j].set_xticklabels(decades_label)

                  for alim in xylim:
                     fig_KS['ax'][i][j].__getattribute__(f'set_{alim}')(xylim[alim])



            if min_axlabels and p_KS[tick][i][j]['meta']['val_key'][:2]!="N_":

               if i!=I_max-1:
                     fig_KS['ax'][i][j].tick_params(labelbottom=False)
                     fig_KS['ax'][i][j].set_xlabel("")

               if j==0:
                     fig_KS['ax'][i][j].set_ylabel(fig_KS['ax'][i][j].get_ylabel().replace(p_KS[tick][i][j]['plot_labels']['xylabel_super'], "\\leftrightharpoons"))
               else:
                     fig_KS['ax'][i][j].tick_params(labelleft=False)
                     fig_KS['ax'][i][j].set_ylabel("")
               
            


   fstmp.fig_export(fig_KS, height=height[I_max])


In [ ]:
tf_mode = 'u'
tf_u_key = 3
min_axlabels = True

periods = ['all']
vals = ['delta_t']
I_s = [[i] for i in range(2)]
Nlim_rf_upper_s = [10,5]
Nlim_rb_upper = 0.01

ylim_abs = 1

height = {1: 2.5, 2: 5, 3: 7}

KS_arrows = [f"\\leftarrow", f"\\rightarrow", f"\\leftrightarrow"]
ineqs = [f"<", f">"]

decades = 10*np.arange(193, 203)
decades_label = [f"{d}" for d in decades]

I_max = 2
J_max = len(I_s)
p_KSLB = {tick: [[None for j in range(J_max)] for i in range(I_max)] for tick in B20.tk}

for tick in B20.tk:

   decades_label = [f"{d}" for d in decades]
   if np.isin(tick, ['SPX', 'DJI']):
        for k in range(decades.size//2):
            decades_label[2*k] = f""

   overwrite_t_ps = {'zoom': ['2008', '2024']}
   tf_args = {'tf_mode': tf_mode}
   if tf_u_key is not None:
        tf_args['tf_u_key'] = tf_u_key
        tf_args['a_u'] = B20.tk[tick]._tail_fit[tf_mode]['lambda']['a_u'][:,tf_u_key]
        tf_args['a_lambda'] = B20.tk[tick]._tail_fit[tf_mode]['lambda']['a_lambda'][:,tf_u_key]
        tf_args['u'] = B20.tk[tick]._tail_fit[tf_mode]['lambda']['u'][:,tf_u_key]

   if tf_mode=='u':
      u_name = tf_args['u']
   else:
      u_name = tf_args['a_u']

   name = f"{tick}_KSLB_{misc_fns.str_concatenate(vals, fill_str='-', fill_str_void=-1)}_{misc_fns.str_concatenate(periods, fill_str='-', fill_str_void=-1)}_U_{tf_mode}={str(np.abs(u_name[0])).replace('.','')}"

   fig_KSLB = plot_fns.rc_plot_setup(name=name, nrows=I_max, ncols=J_max)

   fig_KSLB['ax2'] = [[fig_KSLB['ax'][i][j].twiny() for j in range(len(fig_KSLB['ax'][i]))] for i in range(len(fig_KSLB['ax']))]

   for i in range(I_max):
         for j in range(J_max):

            fig_KSLB['ax2'][i][j].tick_params(left=False, right=False, top=False, bottom=False,
                                             labelleft=False, labelright=False, labeltop=False, labelbottom=False)


   for i in range(I_max):
         for j in range(J_max):
            if i==0:
                p_KSLB[tick][i][j] = lr_fns.KS_plot(self=B20, ax=fig_KSLB['ax'][i][j], ax2=fig_KSLB['ax2'][i][j], tick=tick, period=periods[0], val_key=vals[0], t_bracket=False, thesis=fstmp.get_thesis(),
                                                **tf_args,
                                                I_s=I_s[j],
                                                plot_main_legend=True, plot_ci_legend=(j==0),
                                                overwrite_t_ps=overwrite_t_ps,
                                                Nlim_rf_upper=Nlim_rf_upper_s[i], Nlim_rb_upper=Nlim_rb_upper, 
                                                #an_y0=0.15, 
                                                #an_y0_t=0.21, 
                                                #u_scif_pow10=0,
                                                ci_legend_loc='lower right', ci_legend_ncols=1,
                                                )
            elif i==1:
                p_KSLB[tick][i][j] = lr_fns.LB_plot(self=B20, ax=fig_KSLB['ax'][i][j], ax2=fig_KSLB['ax2'][i][j], tick=tick, period=periods[0], val_key=vals[0], thesis=fstmp.get_thesis(),
                                          **tf_args,
                                          I_s=I_s[j],
                                          plot_main_legend=True, plot_ci_legend=(j==0),
                                          ylim_abs=ylim_abs,
                                          an_v_p='top', an_v_t='bottom', main_legend_loc='lower left', ci_legend_loc='upper right', ci_legend_ncols=1,
                                          #ci_legend_loc='lower right', ci_legend_ncols=1,
                                          #plot_main_legend=False,
                                          #plot_ci_legend=True,
                                          #an_y0_p=0.85, 
                                          #an_y0_t=0.21, 
                                          #u_scif_pow10=0,
                                          #xlabel="",
                                          )



            if min_axlabels:

               if False and i!=I_max-1:
                     fig_KSLB['ax'][i][j].tick_params(labelbottom=False)
                     fig_KSLB['ax'][i][j].set_xlabel("")

               if j==0:
                     fig_KSLB['ax'][i][j].set_ylabel(fig_KSLB['ax'][i][j].get_ylabel().replace(p_KSLB[tick][i][j]['plot_labels']['xylabel_super'], "\\leftrightharpoons"))
               else:
                     fig_KSLB['ax'][i][j].tick_params(labelleft=False)
                     fig_KSLB['ax'][i][j].set_ylabel("")

               




   fstmp.fig_export(fig_KSLB, height=height[I_max])


In [ ]:
tf_mode = 'u'
tf_u_key = 3
min_axlabels = True

periods = ['all']
vals = ['M']
I_s = [[i] for i in range(2)]
Nlim_rf_upper_s = [10,5]
Nlim_rb_upper = 0.01

ylim_abs = 1

height = {1: 2.5, 2: 5, 3: 7}

KS_arrows = [f"\\leftarrow", f"\\rightarrow", f"\\leftrightarrow"]
ineqs = [f"<", f">"]

decades = 10*np.arange(193, 203)
decades_label = [f"{d}" for d in decades]

I_max = 2
J_max = len(I_s)
p_KSLB = {tick: [[None for j in range(J_max)] for i in range(I_max)] for tick in B20.tk}

for tick in B20.tk:

   decades_label = [f"{d}" for d in decades]
   if np.isin(tick, ['SPX', 'DJI']):
        for k in range(decades.size//2):
            decades_label[2*k] = f""

   overwrite_t_ps = {'zoom': ['2008', '2024']}
   tf_args = {'tf_mode': tf_mode}
   if tf_u_key is not None:
        tf_args['tf_u_key'] = tf_u_key
        tf_args['a_u'] = B20.tk[tick]._tail_fit[tf_mode]['lambda']['a_u'][:,tf_u_key]
        tf_args['a_lambda'] = B20.tk[tick]._tail_fit[tf_mode]['lambda']['a_lambda'][:,tf_u_key]
        tf_args['u'] = B20.tk[tick]._tail_fit[tf_mode]['lambda']['u'][:,tf_u_key]

   if tf_mode=='u':
      u_name = tf_args['u']
   else:
      u_name = tf_args['a_u']

   name = f"{tick}_KSLB_{misc_fns.str_concatenate(vals, fill_str='-', fill_str_void=-1)}_{misc_fns.str_concatenate(periods, fill_str='-', fill_str_void=-1)}_U_{tf_mode}={str(np.abs(u_name[0])).replace('.','')}"

   fig_KSLB = plot_fns.rc_plot_setup(name=name, nrows=I_max, ncols=J_max)

   fig_KSLB['ax2'] = [[fig_KSLB['ax'][i][j].twiny() for j in range(len(fig_KSLB['ax'][i]))] for i in range(len(fig_KSLB['ax']))]

   for i in range(I_max):
         for j in range(J_max):

            fig_KSLB['ax2'][i][j].tick_params(left=False, right=False, top=False, bottom=False,
                                             labelleft=False, labelright=False, labeltop=False, labelbottom=False)


   for i in range(I_max):
         for j in range(J_max):
            if i==0:
                p_KSLB[tick][i][j] = lr_fns.KS_plot(self=B20, ax=fig_KSLB['ax'][i][j], ax2=fig_KSLB['ax2'][i][j], tick=tick, period=periods[0], val_key=vals[0], t_bracket=False, thesis=fstmp.get_thesis(),
                                                **tf_args,
                                                I_s=I_s[j],
                                                plot_main_legend=True, plot_ci_legend=(j==0),
                                                overwrite_t_ps=overwrite_t_ps,
                                                Nlim_rf_upper=Nlim_rf_upper_s[i], Nlim_rb_upper=Nlim_rb_upper, 
                                                #an_y0=0.15, 
                                                #an_y0_t=0.21, 
                                                #u_scif_pow10=0,
                                                ci_legend_loc='lower right', ci_legend_ncols=1,
                                                )
            elif i==1:
                p_KSLB[tick][i][j] = lr_fns.LB_plot(self=B20, ax=fig_KSLB['ax'][i][j], ax2=fig_KSLB['ax2'][i][j], tick=tick, period=periods[0], val_key=vals[0], thesis=fstmp.get_thesis(),
                                          **tf_args,
                                          I_s=I_s[j],
                                          plot_main_legend=True, plot_ci_legend=(j==0),
                                          ylim_abs=ylim_abs,
                                          an_v_p='top', an_v_t='bottom', main_legend_loc='lower left', ci_legend_loc='upper right', ci_legend_ncols=1,
                                          #ci_legend_loc='lower right', ci_legend_ncols=1,
                                          #plot_main_legend=False,
                                          #plot_ci_legend=True,
                                          #an_y0_p=0.85, 
                                          #an_y0_t=0.21, 
                                          #u_scif_pow10=0,
                                          #xlabel="",
                                          )



            if min_axlabels:

               if False and i!=I_max-1:
                     fig_KSLB['ax'][i][j].tick_params(labelbottom=False)
                     fig_KSLB['ax'][i][j].set_xlabel("")

               if j==0:
                     fig_KSLB['ax'][i][j].set_ylabel(fig_KSLB['ax'][i][j].get_ylabel().replace(p_KSLB[tick][i][j]['plot_labels']['xylabel_super'], "\\leftrightharpoons"))
               else:
                     fig_KSLB['ax'][i][j].tick_params(labelleft=False)
                     fig_KSLB['ax'][i][j].set_ylabel("")

               




   fstmp.fig_export(fig_KSLB, height=height[I_max])


In [ ]:
fig_KS_demo = plot_fns.rc_plot_setup(name='fig_KS_demo', nrows=2)

fig_KS_demo['ax2'] = [[fig_KS_demo['ax'][i][j].twiny() for j in range(len(fig_KS_demo['ax'][i]))] for i in range(len(fig_KS_demo['ax']))]
for i in range(len(fig_KS_demo['ax'])):
    for j in range(len(fig_KS_demo['ax'][i])):
        fig_KS_demo['ax2'][i][j].xaxis.set_label_position('top')
        fig_KS_demo['ax2'][i][j].tick_params(left=False, right=False, top=False, bottom=False,
                                        labelleft=False, labelright=False, labeltop=False, labelbottom=False)
        fig_KS_demo['ax'][i][j].tick_params(left=True, right=True, top=True, bottom=True,
                                        #labelleft=False, labelright=False, labeltop=False, labelbottom=False
                                        )


an_params = plot_fns.calc_an_params(an_mode='thesis')

T = 20
_seed = 64
xmax = 3.5
xmax_lim = 1.6


#col_F0 = cols[0]
col_F0 = 'black'
col_FK = cols[9]
#cols_dF = cols[2]
col_dF = cols[4]
col_D = cols[5]
col_p = cols[3]

if True:
    T = 30
    _seed = 2888
    xmax_lim = 2.0


CI = [0.95, 0.99]

N_CI = np.size(CI)
if cols_CI is None:
    cols_CI = plot_fns.calc_cols_CI(N_CI=N_CI)



            


xlim = xmax_lim*np.array([-1,1])
ylim = np.array([0,1])

xs = np.linspace(-xmax,xmax, 401)




_unif_rand = scipy.stats.uniform()
_unif_rand.random_state = np.random.Generator(np.random.PCG64(_seed))
F_x = _unif_rand.rvs(T)

x = np.concatenate([[np.nan], np.sort(scipy.stats.norm.ppf(F_x))])
y = np.full(1+T, np.nan)
y[0] = 0
y[1:] = 1+np.arange(T)
y /= T

ys = scipy.stats.norm().cdf(xs)

KS_res = misc_fns.KS_test(x=x[1:], cdf=scipy.stats.norm().cdf, return_full=True)

xs_N = np.concatenate([xs, np.repeat(x[1:],2)])
argsort_xs_N = np.argsort(xs_N)
xs_N = xs_N[argsort_xs_N]
F_N_xs_N = np.concatenate([np.sum(np.greater.outer(xs, x[1:]), axis=1)/T, np.repeat(y,2)[1:-1]])[argsort_xs_N]
F_xs_N = scipy.stats.norm().cdf(xs_N)
diff_F_xs_N = F_N_xs_N - F_xs_N

argmax_abs_diff_F_xs_N = np.argmax(np.abs(diff_F_xs_N))

CI_axis_1 = True

CI_y = np.array([[scipy.stats.norm().cdf(xs) + ((-1)**(1+k))*np.sqrt(np.log(2/((1-CI[j])/2)))/np.sqrt(2*T) for j in range(len(CI))] for k in range(2)])
for i in range(N_CI):
    j = N_CI-(1+i)
    fig_KS_demo['ax'][0][0].fill_between(xs, y1=CI_y[0][j], y2=CI_y[1][j], color=cols_CI[j])
    if CI_axis_1:
        fig_KS_demo['ax'][1][0].fill_between(xs, y1=np.zeros(xs.size), y2=np.sqrt(np.log(2/((1-CI[j])/2)))/np.sqrt(2*T), color=cols_CI[j])

fig_KS_demo['ax'][0][0].plot(xs, ys, color=col_F0, lw=1)
fig_KS_demo['ax'][0][0].plot(np.repeat(x,2)[2:], np.repeat(y,2)[1:-1], color=col_FK, lw=1)


if False:
    fig_KS_demo['ax'][0][0].arrow(x=KS_res[2], dx=0, y=scipy.stats.norm().cdf(KS_res[2]), dy=y[1+KS_res[5]//T]-scipy.stats.norm().cdf(KS_res[2]),
                                    color=cols[5], lw=1, 
                                    #arrowstyle='<->', 
                                        )

    fig_KS_demo['ax'][0][0].arrow(x=xs_N[argmax_abs_diff_F_xs_N], dx=0, y=F_xs_N[argmax_abs_diff_F_xs_N], dy=-diff_F_xs_N[argmax_abs_diff_F_xs_N],
                                    color=cols[5], lw=1, 
                                    #arrowstyle='<->', 
                                        )




fig_KS_demo['ax'][1][0].plot(xs_N, np.abs(diff_F_xs_N), 
                             #color=cols[5], 
                             color=col_dF,
                             lw=1)


    

for ax in ['ax', 'ax2']:
    for i in range(2):
        fig_KS_demo[ax][i][0].set_xlim(xlim)
    fig_KS_demo[ax][0][0].set_ylim(ylim)
    ylim_2 = np.array(fig_KS_demo['ax'][1][0].get_ylim())
    ylim_2[0] = 0
    if False:
        if CI_axis_1:
            ylim_2[1] = 0.2
    fig_KS_demo[ax][1][0].set_ylim(ylim_2)

fig_KS_demo['ax'][-1][0].set_xlabel("Normal quantiles")
fig_KS_demo['ax'][0][0].set_ylabel("$F$")
fig_KS_demo['ax'][1][0].set_ylabel(f"$\\left|F_{{K}} - F_{{0}}\\right|$")

y_arrow = np.array([F_xs_N[argmax_abs_diff_F_xs_N], F_N_xs_N[argmax_abs_diff_F_xs_N]])
for k in range(2):
    fig_KS_demo['ax'][0][0].arrow(x=xs_N[argmax_abs_diff_F_xs_N], dx=0, y=y_arrow[k], dy=((-1)**k)*np.diff(y_arrow)[0],
                                        color=col_D, lw=1, 
                                        length_includes_head=True, 
                                        #head_width=0.03, head_length=0.005,
                                        #head_width=0.0075*np.diff(xlim)[0], head_length=0.03*np.diff(ylim_2)[0],
                                        head_width=1.5*0.0075*np.diff(xlim)[0], head_length=0*0.03*np.diff(ylim_2)[0],
                                        head_starts_at_zero=False,
                                        #arrowstyle='<->', 
                                            )



y_arrow = np.array([0, np.abs(diff_F_xs_N[argmax_abs_diff_F_xs_N])])
for k in range(2):
    fig_KS_demo['ax'][1][0].arrow(x=xs_N[argmax_abs_diff_F_xs_N], dx=0, y=y_arrow[k], dy=((-1)**k)*np.diff(y_arrow)[0],
                                        color=col_D, lw=1, 
                                        length_includes_head=True, 
                                        #head_width=0.03, head_length=0.005,
                                        #head_width=0.0075*np.diff(xlim)[0], head_length=0.03*np.diff(ylim_2)[0],
                                        head_width=1.5*0.0075*np.diff(xlim)[0], head_length=0*0.03*np.diff(ylim_2)[0],
                                        head_starts_at_zero=False,
                                        #arrowstyle='<->', 
                                            )

    
fig_KS_demo['ax'][1][0].annotate(
                                    f"$D_{{K}} = {misc_fns.scif_string(val=y_arrow[1], dp=2, app_dlrs=False)}$",
                                        xy=(xs_N[argmax_abs_diff_F_xs_N]-0.1, y_arrow[1]), xycoords='data',
                                        #textcoords=(xs_N[argmax_abs_diff_F_xs_N]-0.1, y_arrow[1]),
                                        horizontalalignment='right',
                                        verticalalignment='top',
                                        fontsize=an_params['an_fs'],
                                        color=col_D
                                        )
    


xy_box = plot_fns.annotate_pvals(p_vals=[KS_res[1]], p_cols=[col_p], p_symb=[f"p_{{\\mathrm{{KS}}}}"], p_thresh=1-CI[0], ax=fig_KS_demo['ax'][0][0],
                        an_h='left', an_v='top', 
                        an_x0=0.015, an_yi=0.12, flip_x0ry0t=True, an_mode=fstmp.get_an_mode())


bbox = plot_fns.calc_bbox(xy_boxes=[xy_box], along_x=False, along_y=True)
fig_KS_demo['ax'][0][0].legend([fig_KS_demo['ax'][0][0].get_children()[k] for k in N_CI+np.flip(np.arange(2))],
                                    #[f"$N_{{T}}^{{{arrow}}}$" for arrow in arrows],
                                    np.flip([f"$F_{{0}}$", f"$F_{{K}}$"]),
                                    #loc="lower right",
                                    loc="upper left",
                                    bbox_to_anchor=bbox,
                                    fontsize=7)


fig_KS_demo['ax2'][0][0].legend([fig_KS_demo['ax'][0][0].get_children()[N_CI-(1+k)] for k in range(N_CI)], 
                                [f"${100*CI[k]:.0f}\\%$ CI" for k in range(N_CI)],
                                loc='lower right', bbox_to_anchor=bbox, ncol=1, 
                                fontsize=an_params['an_fs'])

fig_KS_demo['ax'][1][0].legend([fig_KS_demo['ax'][1][0].get_children()[int(CI_axis_1*N_CI)+k] for k in [0]],
                                    #[f"$N_{{T}}^{{{arrow}}}$" for arrow in arrows],
                                    [f"$\\left|F_{{K}}-F_{{0}}\\right|$", 
                                     #"$D_{{K}}$"
                                     ],
                                    #loc="lower right",
                                    loc="upper left",
                                    #bbox_to_anchor=bbox,
                                    fontsize=an_params['an_fs'])

fig_KS_demo['fig'].set_size_inches(5.7,3)
fig_KS_demo['fig'].tight_layout()


fstmp.fig_export(fig_KS_demo, height=3)


In [ ]:
plot_np = False
adjusted = False
Q_logscale = False


fig_LB_demo = plot_fns.fig_column_setup(name='fig_LB_demo', hratio=np.array([1,1,2,1]), labeltopbottom=False, tick_direction=None)






T_burn = 1000
T = 100
T_tot = T + T_burn

H = 99
h_s = 1+np.arange(H)


W_beta = 0.4
T_betas = 10


CI = [0.95, 0.99]
N_CI = np.size(CI)
cols_CI = plot_fns.calc_cols_CI(N_CI=N_CI)


#col_F0 = cols[0]
col_F0 = 'black'
col_FK = cols[9]
#cols_dF = cols[2]
col_dF = cols[4]
col_D = cols[5]
col_p = cols[3]



_seed_betas = 34
_unif_rand_betas = scipy.stats.uniform()
_unif_rand_betas.random_state = np.random.Generator(np.random.PCG64(_seed_betas))
F_betas = _unif_rand_betas.rvs(T_betas)
betas = W_beta*scipy.stats.expon(scale=1).pdf(np.arange(T_betas))#*F_betas
print(betas)


alphas = np.array([0.15])
betas = np.array([0.2])


T_alphas = np.size(alphas)
T_betas = np.size(betas)


_seed_epsilon = 12
_unif_rand_epsilon = scipy.stats.uniform()
_unif_rand_epsilon.random_state = np.random.Generator(np.random.PCG64(_seed_epsilon))
F_epsilon = _unif_rand_epsilon.rvs(T_tot)
epsilon = scipy.stats.norm().ppf(F_epsilon)

X = np.full(T_tot, np.nan)
t_s = np.arange(-T_burn, T)

for t in range(T_tot):

    t_0_alpha = np.min([t, T_alphas])
    t_0_beta = np.min([t, T_betas])
        
    X[t] = epsilon[t] + np.sum(alphas[:t_0_alpha]*np.flip(epsilon[t-t_0_alpha:t])) + np.sum(betas[:t_0_beta]*np.flip(X[t-t_0_beta:t]))


fig_LB_demo['ax'][0][0].plot(1+t_s[-T:], X[-T:], color=col_FK)


fig_LB_demo['ax'][0][0].set_xlim([0,T])

fig_LB_demo['ax'][0][0].set_ylim(np.max(np.abs(fig_LB_demo['ax'][0][0].get_ylim()))*np.array([-1,1]))

fig_LB_demo['ax'][0][0].set_ylabel(f"$X_{{k}}$")
fig_LB_demo['ax'][0][0].set_xlabel(f"$k$")


param_str = misc_fns.capp_brac(f"{GR.pltsbv('alpha')}={misc_fns.vector_string(alphas, scif_pow10=0, simple_scalar=False)}, {GR.pltsbv('beta')}={misc_fns.vector_string(betas, scif_pow10=0, simple_scalar=False)}", brac_type="{}", add_lr=True)

plot_fns.annotate_array(ax=fig_LB_demo['ax'][0][0], an_mode=fstmp.get_an_mode(),
                            an_h='right', an_v='bottom',
                            flip_x0ry0t=True,
                            str_list=[f"$K={T}$"])

plot_fns.annotate_array(ax=fig_LB_demo['ax'][0][0], an_mode=fstmp.get_an_mode(),
                            an_h='right', an_v='top', an_yi=0.14,
                            top_to_bottom=True,
                            flip_x0ry0t=True,
                            str_list=[f"$X_{{k}} | \\mathrm{{ARMA}}{{\\left({T_alphas},{T_betas}\\right)}}$",
                                      f"${GR.pltsbv('theta')}_{{\\mathrm{{ARMA}}}} = {param_str}$"])

LB_test = np.transpose(acorr_ljungbox(x=X[T_burn:], lags=H, return_df=False))

rho_sm, Q_sm, p_sm = sm.tsa.stattools.acf(X[T_burn:], nlags=H, qstat=True, adjusted=adjusted)

CI_x = np.arange(2+H)
CI_y = np.array([[((-1)**(1+k))*scipy.stats.norm().ppf((1+CI[j])/2)*np.sqrt(T - CI_x)/np.sqrt((T*(T+2))) for j in range(len(CI))] for k in range(2)])
if False:
    CI_y = np.array([[((-1)**(1+k))*(scipy.stats.norm().ppf((1 + CI[j])/2)/np.sqrt(T - CI_x)) for j in range(len(CI))] for k in range(2)])


for i in range(N_CI):
    j = N_CI-(1+i)
    fig_LB_demo['ax'][1][0].fill_between(CI_x, y1=CI_y[0][j], y2=CI_y[1][j], color=cols_CI[j])

fig_LB_demo['ax'][1][0].bar(h_s, rho_sm[1:], width=0.8, color=col_dF, align='center')
if plot_np:
    rho = misc_fns_nb.calc_autocorrcoef_H(X=X[T_burn:], H=H)
    fig_LB_demo['ax'][1][0].bar(h_s, rho, lw=1, color=cols[1])

if False:
    for i in range(N_CI):
        j = N_CI-(1+i)
        for k in range(2):
            fig_LB_demo['ax'][1][0].plot(h_s,  ((-1)**(1+k))*scipy.stats.norm().ppf((1+CI[j])/2)*np.sqrt(T - h_s)/np.sqrt((T*(T+2))), color='black', lw=0.1)


fig_LB_demo['ax'][1][0].set_xlim([0,H+1])
fig_LB_demo['ax'][1][0].set_ylim(np.array([-1,1]))

fig_LB_demo['ax'][1][0].set_ylabel(f"$\\hat{{\\rho}}_{{h}}$")
fig_LB_demo['ax'][1][0].set_xlabel(f"$h$")

if False:
    plot_fns.annotate_array(ax=fig_LB_demo['ax'][1][0], an_mode=fstmp.get_an_mode(),
                                an_h='left', an_v='bottom',
                                str_list=[f"$H={H}$"])



for i in range(N_CI):
    j = N_CI-(1+i)
    fig_LB_demo['ax'][2][0].fill_between(h_s, y1=np.zeros(H), y2=scipy.stats.chi2(df=h_s).ppf(CI[j]), color=cols_CI[j])



    



dQ_sm = misc_fns.diff_zero(Q_sm)
if Q_logscale:
    Q_sm_m1 = Q_sm[0]*0.9
else:
    Q_sm_m1 = 0.0
Q_sm_m1 = 0.0
dQ_sm_0 = Q_sm[0] - Q_sm_m1

fig_LB_demo['ax'][2][0].bar(h_s, height=np.concatenate([[dQ_sm_0], dQ_sm[1:]]), bottom=np.concatenate([[Q_sm_m1], Q_sm])[:-1], width=0.8, color=col_D, align='center')
if plot_np:
    dQ = T*(T-2)*(rho**2)/(T-h_s)
    Q = np.cumsum(dQ)
    if Q_logscale:
        Q_m1 = Q[0]*0.9
    else:
        Q_m1 = 0.0
    dQ_0 = Q[0] - Q_m1
    fig_LB_demo['ax'][2][0].bar(h_s, height=np.concatenate([[dQ_0], dQ[1:]]), bottom=np.concatenate([[Q_m1], Q])[:-1], width=0.8, color=cols[1], align='center')

if Q_logscale:
    fig_LB_demo['ax'][2][0].set_yscale('log')

fig_LB_demo['ax'][2][0].set_xlim([0,H+1])
fig_LB_demo['ax'][2][0].set_ylim(bottom=Q_sm_m1)
fig_LB_demo['ax'][2][0].set_ylabel(f"$Q_{{H}}$")
fig_LB_demo['ax'][2][0].set_xlabel(f"$H$")






for i in range(N_CI):
    j = N_CI-(1+i)
    fig_LB_demo['ax'][3][0].fill_between(CI_x, y1=np.ones(CI_x.size), y2=np.ones(CI_x.size)*(1-CI[j]), color=cols_CI[j])


fig_LB_demo['ax'][3][0].plot(h_s, p_sm, color=col_p)
if plot_np:
    fig_LB_demo['ax'][3][0].plot(h_s, 1-scipy.stats.chi2(df=h_s).cdf(Q), color=cols[1])

fig_LB_demo['ax'][3][0].set_yscale('log')
fig_LB_demo['ax'][3][0].set_xlim([0,H+1])
fig_LB_demo['ax'][3][0].set_ylim(top=1)

fig_LB_demo['ax'][3][0].set_ylabel(f"$p_{{\\mathrm{{LB}}_{{H}}}}$")
fig_LB_demo['ax'][3][0].set_xlabel(f"$H$")

fig_LB_demo['ax'][3][0].legend([fig_LB_demo['ax'][3][0].get_children()[N_CI-(1+k)] for k in range(N_CI)], 
            [f"${100*CI[k]:.0f}\\%$ CI" for k in range(N_CI)],
            loc='lower left', 
            ncol=2, 
            fontsize=plot_fns.calc_an_params(an_mode=fstmp.get_an_mode())['an_fs'])


fig_LB_demo['fig'].set_size_inches(5.7,7)
fig_LB_demo['fig'].tight_layout()

fstmp.fig_export(fig_LB_demo, height=7)

In [ ]:
def print_table_uv_flatx_ticks(ticks=None, include_pdev=None, do_print=None, do_write=None, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, incl_ticks_column=None, print_ftab=None, ftab_size=None,
                               phantom_sign=None, phantom_brac=None, sign_inside_brac=None, scif_string_format=None):

    if print_ftab is None:
        print_ftab = True

    if include_pdev is None:
        include_pdev = False

    C_str = fstmp.calc_C_str()

    scif_string_spec = {
                        #'exp_zeros': 1,
                        'dp': 1,
                        'phantom_sign': phantom_sign,
                        'phantom_brac': phantom_brac, 
                        'sign_inside_brac': sign_inside_brac, 
                        'scif_string_format': scif_string_format,
                    }   
    scif_string_spec_pd = {
                        'exp_zeros': 0,
                        'dp': 3,
                        'phantom_sign': True, 
                        'phantom_brac': False,
                        'sign_inside_brac': sign_inside_brac, 
                        'scif_string_format': scif_string_format,
                    }  


    tab_str = []
    ftab_str = []
    
    pdev_symb = {
                    'log_L': f"\\ell",
                    'AIC': f"\\mathrm{{AIC}}",
                    'BIC': f"\\mathrm{{BIC}}",
                }
    
    pdev_tab = ['dev', 'AIC', 'BIC']
    
    
    vals = ['est', 'se']
    dists = ['norm', 't']
    params = ['loc', 'scale', 'df']
    params_symb = {
                    'loc': 'mu',
                    'scale': 'sigma',
                    'df': 'nu',
                }
    
    dist_keys = {dist: f'{dist}_ML'for dist in dists}
    
    params_symb_eff = {param: f"\\{params_symb[param]}" for param in params}

    param_index = {
                    'loc': -2,
                    'scale': -1,
                    'df': -3,
                }


    if ticks is None:
        ticks  = np.array([[B20._ticks]])
    ticks = np.array(ticks)




    if incl_ticks_column is None:
        if False:
            if len(np.shape(ticks))<2:
                incl_ticks_column = False
            else:
                incl_ticks_column = True
        else:
            incl_ticks_column = False

    if incl_ticks_column:
        flat_s = 's'
        series_head = "Series & "
        left_r = 2
    else:
        flat_s = 'x'
        series_head = ""
        left_r = 1


    C = len(dists)

    for ticks_t in ticks:

        t_shape = np.array(np.shape(np.array(ticks_t)))
        J = t_shape[1]

        C_J = C*J + (J - 1)

        ticks_t_shape_str = misc_fns.str_concatenate(t_shape, fill_str='x', fill_str_void=-1)
        ticks_t_str = misc_fns.str_concatenate([misc_fns.str_concatenate(misc_fns.make_iterable_array(ticks_t[i])) for i in range(len(ticks_t))], fill_str='x', fill_str_void=-1)

        tab_macro_str = f"TABUvFlat{flat_s}{ticks_t_str}"
        tab_mfn_str = f"\\{C_str['l']}{tab_macro_str}"


        if print_ftab:
            c_str = misc_fns.str_concatenate(np.concatenate([np.repeat('r', left_r), np.repeat('l', C_J)]))
            ftab_str = plot_fns.calc_ftab_str(tab_mfn_str=tab_mfn_str, c_str=c_str, sizes=None, size=ftab_size, ftab_str=ftab_str)

        tab_str.append(f"\\newcommand{{{tab_mfn_str}}}{{")

        for i in range(len(ticks_t)):
            ticks_i = ticks_t[i]

            ticks_i = misc_fns.make_iterable_array(ticks_i)
            tick_0 = ticks_i[0]

            mods = [[B20.tk[tick]._stat_dist_fit[f"{dist}_ML"] for dist in dists] for tick in ticks_i]
            mod_heads = misc_fns.str_concatenate([misc_fns.str_concatenate([f"${B20.tk[tick]._stat_dist_symb_mod[f'{dist}_ML']}$" for dist in dists], fill_str=" & ", fill_str_void=-1) for tick in ticks_i], fill_str=" & & ", fill_str_void=-1)

            if i!=0:
                tab_str.append(f"\\hline")

            if not incl_ticks_column:
                if False:
                    tick_heads = f"\\multicolumn{{{C}}}{{c}}{{{tick.upper()}}}"
                    tick_lines = f"\\cmidrule{{{left_r+1}-{left_r+C}}}"
                else:
                    tick_heads = misc_fns.str_concatenate([f"\\multicolumn{{{C}}}{{c}}{{{tick.upper()}}}" for tick in ticks_i], fill_str=" & & ", fill_str_void=-1)
                    tick_lines = misc_fns.str_concatenate([f"\\cmidrule{{{left_r+1+j*(1+C)}-{left_r+C+j*(1+C)}}}" for j in range(J)])

                tab_str.append(f" & {tick_heads} \\\\")
                tab_str.append(tick_lines)

            tab_str.append(f"{series_head}$\\hat{{{GR.sbv('theta')}}}$ & {mod_heads} \\\\")
            tab_str.append(f"\\hline")

            for param in params:
                #param_val = [[mods[m][val] for val in ['est', 'se']] for m in range(len(mods))]
                param_val = np.full((J, C, len(vals)), np.nan)
                param_p = [[None for m in range(C)] for t in range(J)]

                for j in range(J):
                    for m in range(C):
                        if Tmisc.get_sd_not_scale() and dists[m]=='t':
                            nu = mods[j][m]['est'][param_index['df']]
                            factor = np.sqrt(nu/(nu-2))
                        else:
                            factor = 1


                        for k in range(len(vals)):
                            if np.isin(param, dist_fns.get_scipy_dist_param_list(dist=dists[m])):
                                param_val[j][m][k] = factor*mods[j][m][vals[k]][param_index[param]]
                            if param_val[j][m][k] is None:
                                param_val[j][m][k] = 0
                            #elif param=='mu' or param=='omega':
                            #    param_val[m][k] /= B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist]._scale

                        if np.isnan(param_val[j][m][0]):
                            param_p[j][m] = ""
                        else:
                            param_p[j][m] = misc_fns.scif_string(val=param_val[j][m][0], se=param_val[j][m][1], **scif_string_spec)

                if incl_ticks_column:
                    if param==params[0]:
                        tick_str = f"{ticks_i[0].upper()} & "
                    else:
                        tick_str = f" & "
                else:
                    tick_str = f""
                
                param_str = misc_fns.str_concatenate([misc_fns.str_concatenate(param_p[j], fill_str=" & ", fill_str_void=-1) for j in range(J)], fill_str=" & & ", fill_str_void=-1)
                tab_str.append(f"{tick_str}${params_symb_eff[param]}$ & {param_str} \\\\")
            

            if include_pdev:
                tab_str.append(f"\\hdashline")


                if incl_ticks_column:
                    tick_str = f" & "
                else:
                    tick_str = f""
                T = np.array([[mods[j][m]['eval']['N_data'] for m in range(C)] for j in range(J)])


                pdev_symb_eff = {pdv: f"{pdev_symb[pdv]}/\\Delta{{T}}" for pdv in pdev_symb}
                pdev_symb_eff['dev'] = f"-2{pdev_symb_eff['log_L']}"


                pdev = {pdv: np.array([[mods[j][m]['eval'][pdv]/T[j][m] for m in range(C)] for j in range(J)]) for pdv in pdev_symb}
                pdev['dev'] = -2*pdev['log_L']

                pdev_str = {pdv: misc_fns.str_concatenate([misc_fns.str_concatenate([misc_fns.scif_string(pdev[pdv][j][m], **scif_string_spec_pd)for m in range(C)], fill_str=" & ", fill_str_void=-1) for j in range(J)], fill_str=" & & ", fill_str_void=-1) for pdv in pdev_tab}


                for pdv in pdev_str:
                    tab_str.append(f"{tick_str}${pdev_symb_eff[pdv]}$ & {pdev_str[pdv]} \\\\")


        tab_str.append(f"}}")
        tab_str.append(f"")


        if file_nm is None:
            file_nm = "uv_flatx_ticks"

        fstmp.print_str_pw(print_str=tab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"tab_{file_nm}", file_ext=file_ext)
        if print_ftab:
            fstmp.print_str_pw(print_str=ftab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"ftab_{file_nm}", file_ext=file_ext)



In [ ]:
phantom_sign=True
phantom_brac=True
sign_inside_brac=None
scif_string_format='E'
include_pdev = False
ftab_size=None

print_table_uv_flatx_ticks(ticks=np.moveaxis(np.array([[['SPX', 'DJI', 'NKX'], ['DAX', 'CAC', 'HSI']]]), source=-2, destination=-1), include_pdev=include_pdev, do_print=True, do_write=True, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, ftab_size=ftab_size,
                           phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

In [ ]:
def print_table_cv_flats_ticks(ticks=None, include_pdev=None, do_print=None, do_write=None, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, incl_ticks_column=None, print_ftab=None, ftab_size=None,
                               phantom_sign=None, phantom_brac=None, sign_inside_brac=None, scif_string_format=None):

    if print_ftab is None:
        print_ftab = True

    if include_pdev is None:
        include_pdev = False

    C_str = fstmp.calc_C_str()

    scif_string_spec = {
                        #'exp_zeros': 1,
                        'dp': 1,
                        'phantom_sign': phantom_sign, 
                        'phantom_brac': phantom_brac,
                        'sign_inside_brac': sign_inside_brac, 
                        'scif_string_format': scif_string_format,
                    }   
    scif_string_spec_pd = {
                        'exp_zeros': 0,
                        'dp': 3,
                        'phantom_sign': True, 
                        'phantom_brac': False,
                        'sign_inside_brac': sign_inside_brac, 
                        'scif_string_format': scif_string_format,
                    }  
    
    tab_str = []
    ftab_str = []

    symbols =   {
                'mu': 'mu',
                'omega': 'omega',
                'alpha_1': 'alpha_1',
                'beta_1': 'beta_1',
                'gamma_1': 'gamma_1',
                'df': 'nu',
                'nc': 'lambda',
            }
    
    pdev_symb = {
                    'log_L': f"\\ell",
                    'AIC': f"\\mathrm{{AIC}}",
                    'BIC': f"\\mathrm{{BIC}}",
                }
    
    pdev_tab = ['dev', 'AIC', 'BIC']
    
    

    vol='GARCH'
    p=1
    q=1
    o_s = np.arange(2)
    dists = ['norm', 't']
    params = ['mu', 'omega', 'alpha_1', 'beta_1', 'gamma_1', 'df', 
            #'nc'
            ]
    params_symb = {param: f"\\{param}" for param in params}
    params_symb['df'] = f"\\nu"

    if False:
        N_params = np.concatenate([
                        o+np.array([4,5]) for o in o_s
                    ])
    else:
        N_params = np.array([
                        4+int(dist=='t')+o for dist in dists for o in o_s
                    ])
        

    #params_symb['nc'] = f"\\lambda"

    if ticks is None:
        ticks  = np.array([B20._ticks])
    ticks = np.array(ticks)


    if incl_ticks_column is None:
        if False:
            if len(np.shape(ticks))<2:
                incl_ticks_column = False
            else:
                incl_ticks_column = True
        else:
            incl_ticks_column = False

    if incl_ticks_column:
        flat_s = 's'
        series_head = "Series & "
        left_r = 2
    else:
        flat_s = ''
        series_head = ""
        left_r = 1

    C = len(dists)*len(o_s)
    

    for ticks_i in ticks:

        

        ticks_i = misc_fns.make_iterable_array(ticks_i)
        ticks_i_str = misc_fns.str_concatenate(ticks_i)

        tab_macro_str = f"TABGarchFlat{flat_s}{ticks_i_str.upper()}"
        tab_mfn_str = f"\\{C_str['l']}{tab_macro_str}"


        if print_ftab:
            c_str = misc_fns.str_concatenate(np.concatenate([np.repeat('r', left_r), np.repeat('l', C)]))
            ftab_str = plot_fns.calc_ftab_str(tab_mfn_str=tab_mfn_str, c_str=c_str, sizes=None, size=ftab_size, ftab_str=ftab_str)
            

        tab_str.append(f"\\newcommand{{{tab_mfn_str}}}{{")


        for tick in ticks_i:
            mods = [B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist] for dist in dists for o in o_s]
            M = len(mods)
            mod_heads = misc_fns.str_concatenate([f"${cvol_fns.calc_cv_symb(vol=vol, o=mods[m]._o, dist=mods[m]._dist, tex=True)}$" for m in range(M)], fill_str=" & ", fill_str_void=-1)

            if tick!=ticks_i[0]:
                tab_str.append(f"\\hline")

            if not incl_ticks_column:
                tab_str.append(f" & \\multicolumn{{{C}}}{{c}}{{{tick.upper()}}} \\\\")
                tab_str.append(f"\\cmidrule{{{left_r+1}-{left_r+C}}}")

            tab_str.append(f"{series_head}$\\hat{{{GR.sbv('theta')}}}$ & {mod_heads} \\\\")
            tab_str.append(f"\\hline")
            colvals = [
                        {
                            'tick': tick,
                            'o': mods[m]._o,
                            'dist': mods[m]._dist,
                        }
                        for m in range(len(mods))]
            for param in params:
                param_val = [[misc_fns.index_df(B20.tk[tick].cv._rw['fit']['w']['cv'][val], colval=colvals[m])[param].values[0] for val in ['est', 'se']] for m in range(len(mods))]
                param_p = [None for m in range(len(param_val))]

                

                for m in range(len(param_val)):

                    if Tmisc.get_sd_not_scale() and colvals[m]['dist']=='t' and np.isin(param, ['omega']):
                        nu = misc_fns.index_df(B20.tk[tick].cv._rw['fit']['w']['cv']['est'], colval=colvals[m])['df'].values[0]
                        factor = np.sqrt((nu)/(nu-2))
                    else:
                        factor = 1
                    if True:
                        if param=='omega': 
                            scale = mods[m]._scale
                            param_val[m][0] = np.sqrt(param_val[m][0])/scale
                            param_val[m][1] = param_val[m][1]/(2*param_val[m][0]*(scale**2))
                    for k in range(2):
                        if param_val[m][k] is None:
                            param_val[m][k] = 0

                        param_val[m][k] *= factor

                    if np.isnan(param_val[m][0]):
                        param_p[m] = ""
                    else:
                        param_p[m] = misc_fns.scif_string(val=param_val[m][0], se=param_val[m][1], **scif_string_spec)

                if incl_ticks_column:
                    if param==params[0]:
                        tick_str = f"{tick.upper()} & "
                    else:
                        tick_str = f" & "
                else:
                    tick_str = f""
                
                param_symb = f"$\\{symbols[param]}$"
                param_str = misc_fns.str_concatenate(param_p, fill_str=" & ", fill_str_void=-1)
                tab_str.append(f"{tick_str}{param_symb} & {param_str} \\\\")


            if include_pdev:
                tab_str.append(f"\\hdashline")


                if incl_ticks_column:
                    tick_str = f" & "
                else:
                    tick_str = f""
                T = np.diff(B20.tk[tick].pd.get_periods_t_offset()['train'])[0]
                #log_L =  np.array([misc_fns.index_df(B20.tk[tick].cv._rw['fit']['w']['cv']['est'], colval=colvals[m])['log_L'].values[0] for m in range(len(mods))])
                log_L =  np.array([np.sum(np.log(mods[m].f_X)) for m in range(len(mods))])

                


                pdev_symb_eff = {pdv: f"{pdev_symb[pdv]}/\\Delta{{T}}" for pdv in pdev_symb}
                pdev_symb_eff['dev'] = f"-2{pdev_symb_eff['log_L']}"


                pdev = {
                            'log_L': log_L/T,
                            'AIC': np.array([misc_fns.calc_AIC(log_L=log_L[m], N_params=N_params[m]) for m in range(len(mods))])/T,
                            'BIC': np.array([misc_fns.calc_BIC(log_L=log_L[m], N_params=N_params[m], N_data=T) for m in range(len(mods))])/T,
                        }
                pdev['dev'] = -2*pdev['log_L']

                pdev_str = {pdv: misc_fns.str_concatenate([
                                                           #f"${:.3f}$" 
                                                            misc_fns.scif_string(pdev[pdv][m], **scif_string_spec_pd)
                                                           for m in range(len(mods))], fill_str=" & ", fill_str_void=-1) for pdv in pdev_tab}


                for pdv in pdev_str:
                    tab_str.append(f"{tick_str}${pdev_symb_eff[pdv]}$ & {pdev_str[pdv]} \\\\")


        tab_str.append(f"}}")
        tab_str.append(f"")


    if file_nm is None:
        file_nm = "cv_flats_ticks"

    fstmp.print_str_pw(print_str=tab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"tab_{file_nm}", file_ext=file_ext)
    if print_ftab:
        fstmp.print_str_pw(print_str=ftab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"ftab_{file_nm}", file_ext=file_ext)

In [ ]:
phantom_sign=True
phantom_brac=True
sign_inside_brac=None
scif_string_format='E'
include_pdev = False
ftab_size=None

print_table_cv_flats_ticks(ticks=None, include_pdev=include_pdev, do_print=True, do_write=True, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, ftab_size=ftab_size,
                           phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

print_table_cv_flats_ticks(ticks=np.array([['SPX', 'DJI', 'NKX'], ['DAX', 'CAC', 'HSI']]), include_pdev=include_pdev, do_print=True, do_write=True, write_mode='a', write_address=None, root=None, file_nm=None, file_ext=None, ftab_size=ftab_size,
                           phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

print_table_cv_flats_ticks(ticks=B20._ticks, include_pdev=include_pdev, do_print=True, do_write=True, write_mode='a', write_address=None, root=None, file_nm=None, file_ext=None, ftab_size=ftab_size,
                           phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

In [ ]:
def print_table_ucv_LR_ticks(ticks=None, periods=None, period_head_dict=None, do_print=None, do_write=None, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, print_ftab=None, ftab_size=None, 
                             phantom_sign=None, phantom_brac=None, sign_inside_brac=None, scif_string_format=None):

    if print_ftab is None:
        print_ftab = True


    C_str = fstmp.calc_C_str()


    tab_str = []
    ftab_str = []

    if ticks is None:
        ticks  = np.array([B20._ticks])
    ticks = np.array(ticks)


    if periods is None:
        periods = ['train', 'forecast']
    periods = misc_fns.make_iterable_array(periods)

    vol = 'GARCH'
    p = 1
    q = 1
    o_s = np.arange(2)
    dists = ['norm', 't']

    ticks_flat = misc_fns.make_iterable_array(ticks)

    mods_N_params = {f"G1{o}1-{dist}": 4+int(dist=='t')+o for dist in dists for o in o_s}
    mods_symb = {f"G1{o}1-{dist}": f"${cvol_fns.calc_cv_symb(vol=vol, o=o, dist=dist, tex=True)}$" for dist in dists for o in o_s}

    log_L_t = {tick: {f"G1{o}1-{dist}": np.log(B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist]._bdist.pdf(B20.tk[tick].pd.lr['p']['all'].values)) for dist in dists for o in o_s} for tick in ticks_flat}
    for dist in dists:
        mods_N_params[f'G000-{dist}'] = 2+int(dist=='t')
        mods_symb[f'G000-{dist}'] = f"${cvol_fns.calc_uv_symb(dist=dist, tex=True)}$"
        for tick in ticks_flat:
            log_L_t[tick][f'G000-{dist}'] = np.log(B20.tk[tick]._stat_dist[f'{dist}_ML'].pdf(B20.tk[tick].pd.lr['p']['all'].values))


    periods_t = {tick: B20.tk[tick].pd.get_periods_t_offset(forecast_as_list=False) for tick in ticks_flat}
    log_L = {tick: {mod: {period:np.sum(log_L_t[tick][mod][periods_t[tick][period][0]:periods_t[tick][period][1]]) for period in periods_t[tick]} for mod in log_L_t[tick]} for tick in ticks_flat}

    smods_N_params = misc_fns.d_sort_d(d=mods_N_params, ret=None)
    smods_keys = misc_fns.d_keys(smods_N_params)


    Q = np.add.outer(misc_fns.d_vals(smods_N_params), -misc_fns.d_vals(smods_N_params))
    smods_N_params_v = misc_fns.d_vals(smods_N_params)
    smods_N_params_v_unique = np.unique(smods_N_params_v)

    mod_pairs = {d: np.where(np.tril(Q)[d,:]>0)[0] for d in np.where(smods_N_params_v>smods_N_params_v_unique[0])[0]}
    mod_pairs_key = {smods_keys[d_0]: [smods_keys[d_1] for d_1 in mod_pairs[d_0]] for d_0 in mod_pairs}

    mod_pairs_lite = {d: np.where(smods_N_params_v==smods_N_params_v_unique[np.where(smods_N_params_v[d]==smods_N_params_v_unique)[0]-1])[0] for d in np.where(smods_N_params_v>smods_N_params_v_unique[0])[0]}
    mod_pairs_lite_key = {smods_keys[d_0]: [smods_keys[d_1] for d_1 in mod_pairs_lite[d_0]] for d_0 in mod_pairs_lite}

    mod_pairs_res = {tick:
                {key_0:
                {key_1:
                {period: 
                misc_fns.LR_test_dict(D_0=mods_N_params[key_0], D_1=mods_N_params[key_1], log_L_0=log_L[tick][key_0][period], log_L_1=log_L[tick][key_1][period], incl_p_str=True, p_str_thresh_format='HL', p_thresh=None, 
                                      phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)
                for period in log_L[tick][key_0]}
                for key_1 in mod_pairs_key[key_0]}
                for key_0 in mod_pairs_key}
                for tick in ticks_flat}


    left_r = 2

    P = len(periods)

    for ticks_ij in ticks:

        ticks_shape = np.array(np.shape(ticks_ij))
        T0 = ticks_shape[0]
        T1 = ticks_shape[1]

        if T1>=6:
            period_same_p_rows = False
        else:
            period_same_p_rows = True

        
        if period_same_p_rows:
            metaperiods = [[periods[p] for p in range(P)]]
        else:
            metaperiods = [[periods[p]] for p in range(P)]


        metaP = len(metaperiods)
        subP = len(metaperiods[0])
        C = subP*(1+T1) - 1

        lead = misc_fns.str_concatenate([f"& " for i in range(2)])
        period_head = B20.calc_table_periods_head(tick=None, periods=periods, pre=None, post=None, sample_labels=None, q_same_row=~period_same_p_rows, p_same_rows=period_same_p_rows, N_p_max=None, lead=lead, c0=None, cw=T1, calign='c')

        #ticks_i = misc_fns.make_iterable_array(ticks_i)
        ticks_ij_str = misc_fns.str_concatenate([misc_fns.str_concatenate(misc_fns.array_attr(a=ticks_ij[i], attr='upper')) for i in range(ticks_shape[0])], fill_str='x', fill_str_void=-1)

        tab_macro_str = f"TABUcvLR{ticks_ij_str}"
        tab_mfn_str = f"\\{C_str['l']}{tab_macro_str}"


        
        if print_ftab:
            c_str = misc_fns.str_concatenate(np.concatenate([np.repeat('r', left_r), np.repeat('l', C)]))
            ftab_str = plot_fns.calc_ftab_str(tab_mfn_str=tab_mfn_str, c_str=c_str, sizes=None, size=ftab_size, ftab_str=ftab_str)

        
        

        tab_str.append(f"\\newcommand{{{tab_mfn_str}}}{{")

        for mp in range(metaP):
            for i in range(T0):

                if not (i==0 and mp==0):
                    tab_str.append(f"\\hline")

                tab_str.append(f"{lead}\\multicolumn{{{C}}}{{c}}{{$p_{{\\mathrm{{LR}}}}$}}")
                tab_str.append(f"\\\\")
                tab_str.append(f"\\cmidrule{{{1+left_r}-{left_r+C}}}")
                tab_str.append(period_head[mp])
                tick_label_str = [misc_fns.str_concatenate([f'\\multicolumn{{{1}}}{{c}}{{{tick.upper()}}}' for tick in ticks_ij[i,:]], fill_str=' & ', fill_str_void=-1) for k in range(subP)]
                tab_str.append(f"${GR.sbv('theta')}_{{0}}$ & ${GR.sbv('theta')}_{{1}}$ & {misc_fns.str_concatenate(tick_label_str, fill_str=' & & ', fill_str_void=-1)} \\\\")
                tab_str.append(f"\\hline")
                for key_0 in mod_pairs_lite_key:
                    for key_1 in mod_pairs_lite_key[key_0]:
                        res_str = [misc_fns.str_concatenate([mod_pairs_res[tick][key_0][key_1][period]['p_str_thresh'] for tick in ticks_ij[i]], fill_str=' & ', fill_str_void=-1) for period in metaperiods[mp]]
                        tab_str.append(f"{mods_symb[key_0]} & {mods_symb[key_1]} & {misc_fns.str_concatenate(res_str, fill_str=' & & ', fill_str_void=-1)} \\\\")




        tab_str.append(f"}}")
        tab_str.append(f"")


    if file_nm is None:
        file_nm = "ucv_LR_ticks"

    fstmp.print_str_pw(print_str=tab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"tab_{file_nm}", file_ext=file_ext)
    if print_ftab:
        fstmp.print_str_pw(print_str=ftab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"ftab_{file_nm}", file_ext=file_ext)



In [ ]:
ticks_2D = np.array([[['SPX', 'DJI', 'NKX'], ['DAX', 'CAC', 'HSI']]])
periods = ['train']
phantom_sign=False
phantom_brac=False
sign_inside_brac=None
scif_string_format='E'
ftab_size=None


print_table_ucv_LR_ticks(ticks=ticks_2D, periods=periods, do_print=True, do_write=True, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, ftab_size=ftab_size,
                         phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

print_table_ucv_LR_ticks(ticks=np.moveaxis(ticks_2D, source=-1, destination=-2), periods=periods, do_print=True, do_write=True, write_mode='a', write_address=None, root=None, file_nm=None, file_ext=None, ftab_size=ftab_size,
                         phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

print_table_ucv_LR_ticks(ticks=np.array([[B20._ticks]]), periods=periods, do_print=True, do_write=True, write_mode='a', write_address=None, root=None, file_nm=None, file_ext=None, ftab_size=ftab_size,
                         phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

In [ ]:
def print_table_ucv_PD_ticks(ticks=None, periods=None, period_head_dict=None, do_print=None, do_write=None, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, print_ftab=None, ftab_size=None, 
                             phantom_sign=None, phantom_brac=None, sign_inside_brac=None, scif_string_format=None):

    if print_ftab is None:
        print_ftab = True

    C_str = fstmp.calc_C_str()


    tab_str = []
    ftab_str = []

    if ticks is None:
        ticks  = np.array([B20._ticks])
    ticks = np.array(ticks)


    if periods is None:
        periods = ['train', 'forecast']
    periods = misc_fns.make_iterable_array(periods)

    vol = 'GARCH'
    p = 1
    q = 1
    o_s = np.arange(2)
    dists = ['norm', 't']

    PDs = ['dev', 'AIC', 'BIC']
    PDs = misc_fns.make_iterable_array(PDs)
    symb_PD = { 
                    'dev': f"-2\ell",
                    'AIC': f"\\mathrm{{AIC}}",
                    'BIC': f"\\mathrm{{BIC}}",
                }

    ticks_flat = misc_fns.make_iterable_array(ticks)

    mods_N_params = {f"G1{o}1-{dist}": 4+int(dist=='t')+o for dist in dists for o in o_s}
    mods_symb = {f"G1{o}1-{dist}": f"${cvol_fns.calc_cv_symb(vol=vol, o=o, dist=dist, tex=True)}$" for dist in dists for o in o_s}

    log_L_t = {tick: {f"G1{o}1-{dist}": np.log(B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist]._bdist.pdf(B20.tk[tick].pd.lr['p']['all'].values)) for dist in dists for o in o_s} for tick in ticks_flat}
    for dist in dists:
        mods_N_params[f'G000-{dist}'] = 2+int(dist=='t')
        mods_symb[f'G000-{dist}'] = f"${cvol_fns.calc_uv_symb(dist=dist, tex=True)}$"
        for tick in ticks_flat:
            log_L_t[tick][f'G000-{dist}'] = np.log(B20.tk[tick]._stat_dist[f'{dist}_ML'].pdf(B20.tk[tick].pd.lr['p']['all'].values))


    periods_t = {tick: B20.tk[tick].pd.get_periods_t_offset(forecast_as_list=False) for tick in ticks_flat}
    N_data = {tick: {period: np.diff(periods_t[tick][period])[0] for period in periods_t[tick]} for tick in periods_t}
    log_L = {tick: {mod: {period:np.sum(log_L_t[tick][mod][periods_t[tick][period][0]:periods_t[tick][period][1]]) for period in periods_t[tick]} for mod in log_L_t[tick]} for tick in ticks_flat}

    smods_N_params = misc_fns.d_sort_d(d=mods_N_params, ret=None)
    smods_keys = misc_fns.d_keys(smods_N_params)


    Q = np.add.outer(misc_fns.d_vals(smods_N_params), -misc_fns.d_vals(smods_N_params))
    smods_N_params_v = misc_fns.d_vals(smods_N_params)
    smods_N_params_v_unique = np.unique(smods_N_params_v)

    mod_pairs = {d: np.where(np.tril(Q)[d,:]>0)[0] for d in np.where(smods_N_params_v>smods_N_params_v_unique[0])[0]}
    mod_pairs_key = {smods_keys[d_0]: [smods_keys[d_1] for d_1 in mod_pairs[d_0]] for d_0 in mod_pairs}

    mod_pairs_lite = {d: np.where(smods_N_params_v==smods_N_params_v_unique[np.where(smods_N_params_v[d]==smods_N_params_v_unique)[0]-1])[0] for d in np.where(smods_N_params_v>smods_N_params_v_unique[0])[0]}
    mod_pairs_lite_key = {smods_keys[d_0]: [smods_keys[d_1] for d_1 in mod_pairs_lite[d_0]] for d_0 in mod_pairs_lite}



    mod_res = {tick:
                {key_0:
                {period: 
                misc_fns.PD_dict(N_params=mods_N_params[key_0], 
                                        N_data=N_data[tick][period],
                                      log_L=log_L[tick][key_0][period], 
                                      T=N_data[tick][period],
                                    dp=3, dp_intr=None, sci_pow10=None, sci_pow10_intr=0, symb_pre=None, symb_post=f"_{{r}}", intr_symb=f"\\Delta{{T}}",  
                                    incl_str=True, 
                                    phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)
                for period in log_L[tick][key_0]}
                for key_0 in smods_keys}
                for tick in ticks_flat}


    left_r = 2

    P = len(periods)

    for ticks_ij in ticks:

        ticks_shape = np.array(np.shape(ticks_ij))
        T0 = ticks_shape[0]
        T1 = ticks_shape[1]


        

        if T1>=6:
            period_same_p_rows = False
        else:
            period_same_p_rows = True

        
        if period_same_p_rows:
            metaperiods = [[periods[p] for p in range(P)]]
        else:
            metaperiods = [[periods[p]] for p in range(P)]


        metaP = len(metaperiods)
        subP = len(metaperiods[0])
        C = subP*(1+T1) - 1
            

        lead = misc_fns.str_concatenate([f"& " for i in range(2)])
        period_head = B20.calc_table_periods_head(tick=None, periods=periods, pre=None, post=None, sample_labels=None, q_same_row=~period_same_p_rows, p_same_rows=period_same_p_rows, N_p_max=None, lead=lead, c0=None, cw=T1, calign='c', return_heads=False)


        #ticks_i = misc_fns.make_iterable_array(ticks_i)
        ticks_ij_str = misc_fns.str_concatenate([misc_fns.str_concatenate(misc_fns.array_attr(a=ticks_ij[i], attr='upper')) for i in range(ticks_shape[0])], fill_str='x', fill_str_void=-1)

        tab_macro_str = f"TABUcvPD{ticks_ij_str}"
        tab_mfn_str = f"\\{C_str['l']}{tab_macro_str}"


        
        if print_ftab:
            c_str = misc_fns.str_concatenate(np.concatenate([np.repeat('r', left_r), np.repeat('l', C)]))
            ftab_str = plot_fns.calc_ftab_str(tab_mfn_str=tab_mfn_str, c_str=c_str, sizes=None, size=ftab_size, ftab_str=ftab_str)

        
        

        tab_str.append(f"\\newcommand{{{tab_mfn_str}}}{{")

        for mp in range(metaP):
            for i in range(T0):

                if not (mp==0 and i==0):
                    tab_str.append(f"\\hline")

                tab_str.append(f"{lead}\\multicolumn{{{C}}}{{c}}{{$\\mathrm{{PD}}/\Delta{{T}}_{{p}}$}}")
                tab_str.append(f"\\\\")
                tab_str.append(f"\\cmidrule{{{1+left_r}-{left_r+C}}}")
                tab_str.append(period_head[mp])
                tick_label_str = [misc_fns.str_concatenate([f'\\multicolumn{{{1}}}{{c}}{{{tick.upper()}}}' for tick in ticks_ij[i,:]], fill_str=' & ', fill_str_void=-1) for k in range(subP)]
                tab_str.append(f"$\\mathrm{{PD}}$ & ${GR.sbv('theta')}$ & {misc_fns.str_concatenate(tick_label_str, fill_str=' & & ', fill_str_void=-1)} \\\\")
                tab_str.append(f"\\hline")
                for l0 in range(len(PDs)):
                    PD = PDs[l0]
                    if l0!=0:
                        tab_str.append(f"\\hdashline")

                    tab_str.append(f"${symb_PD[PD]}_{{r}}$")
                    for l1 in range(len(smods_keys)):
                        key_0 = smods_keys[l1]
                        tab_str.append(f"& {mods_symb[key_0]} ")
                    
                        res_str = [misc_fns.str_concatenate([mod_res[tick][key_0][period]['val_intr_str'][PD] for tick in ticks_ij[i]], fill_str=' & ', fill_str_void=-1) for period in metaperiods[mp]]
                        tab_str.append(f"& {misc_fns.str_concatenate(res_str, fill_str=' & & ', fill_str_void=-1)}")
                        tab_str.append(f"\\\\")




        tab_str.append(f"}}")
        tab_str.append(f"")


    if file_nm is None:
        file_nm = "ucv_PD_ticks"

    fstmp.print_str_pw(print_str=tab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"tab_{file_nm}", file_ext=file_ext)
    if print_ftab:
        fstmp.print_str_pw(print_str=ftab_str, do_print=do_print, do_write=do_write, write_mode=write_mode, write_address=write_address, root=root, file_nm=f"ftab_{file_nm}", file_ext=file_ext)



In [ ]:
ticks_2D = np.array([[['SPX', 'DJI', 'NKX'], ['DAX', 'CAC', 'HSI']]])
periods = ['train']
phantom_sign=True
phantom_brac=False
sign_inside_brac=None
scif_string_format='E'
ftab_size=None


print_table_ucv_PD_ticks(ticks=ticks_2D, periods=periods, do_print=True, do_write=True, write_mode=None, write_address=None, root=None, file_nm=None, file_ext=None, ftab_size=ftab_size, 
                         phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

print_table_ucv_PD_ticks(ticks=np.moveaxis(ticks_2D, source=-1, destination=-2), periods=periods, do_print=True, do_write=True, write_mode='a', write_address=None, root=None, file_nm=None, file_ext=None, ftab_size=ftab_size, 
                         phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

print_table_ucv_PD_ticks(ticks=np.array([[B20._ticks]]), periods=periods, do_print=True, do_write=True, write_mode='a', write_address=None, root=None, file_nm=None, file_ext=None, ftab_size=ftab_size, 
                         phantom_sign=phantom_sign, phantom_brac=phantom_brac, sign_inside_brac=sign_inside_brac, scif_string_format=scif_string_format)

In [ ]:
a_u_evt = np.linspace(0.0025, 0.25, 101)
a_u_evt

In [ ]:
a_u_evt = np.linspace(0.0025, 0.25, 101)

a_u_evt_2T = np.array([a_u_evt for k in range(2)])

u_cv_evt = np.outer([-1,1], np.linspace(0.66, 3, 118))


for tick in B20.tk:

    train_t = B20.tk[tick].pd.get_periods_t_offset()['train']
    for vol in B20.tk[tick].cv.cv.cv_v:
        for p in B20.tk[tick].cv.cv.cv_v[vol].cv_p:
            for o in B20.tk[tick].cv.cv.cv_v[vol].cv_p[p]:
                for q in B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o]:
                    for dist in B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d:

                        X = B20.tk[tick].pd.lr["p"]["all"].values
                        mu = B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist]._bdist.kwds['loc']
                        sigma = B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist]._bdist.kwds['scale']

                        u_cv_evt_a_u = B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist]._tdist.ppf([a_u_evt, 1-a_u_evt])
                        hat_epsilon = ((X-mu)/sigma)[train_t[0]:train_t[1]]

                        B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist]._tail_fit = dict()
                        
                        B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist]._tail_fit['u_fine'] = dist_fns.calc_tail_fit(hat_epsilon, u=u_cv_evt)
                        B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist]._tail_fit['a_u_fine'] = dist_fns.calc_tail_fit(hat_epsilon, u=u_cv_evt_a_u, a_u=a_u_evt_2T)  

In [ ]:
def plot_U_u_key_cv(ticks=None, u_key=None, incl_N_ax2=None, split_arrow_legend=None, tick_top=None, xlim=None, xticks=None, height=None, width=None): 

    if u_key is None:
        u_key = 'u'

    if width is None:
        width = 5.7

    if height is None:
        height = 7

    vol='GARCH'
    p=1
    q=1
    o_s = np.arange(2)
    dists = ['norm', 't']

    
    an_params = plot_fns.calc_an_params(an_mode='thesis')

    if incl_N_ax2 is None:
        incl_N_ax2 = False

    if split_arrow_legend is None:
        split_arrow_legend = True

    if tick_top is None:
        tick_top = True

    if ticks is None:
        ticks = [B20._ticks]
    ticks_flat = misc_fns.make_iterable_array(ticks)
    T = {tick: B20.tk[tick]._T['all'] for tick in ticks_flat}

    arrows = [f'\\leftarrow', f'\\rightarrow']
    if xlim is None:
        if u_key=='u':
            xlim = [0.65, 3.01]
        else:
            #xlim=[0,0.25]
            xlim=[0.0025,0.25]


    if u_key=='u':
        I_max = 5
        logscale_i = np.concatenate([[0]])
        logscale_j = False
        arrow_v = 'upper'
        arrow_h = 'left'
        mod_v = 'bottom'
        
    else:
        I_max = 5
        #logscale_i = np.concatenate([[0], I_max-np.array([4,2])])
        logscale_i = np.concatenate([[]])
        #arrow_v = 'lower'
        arrow_v = 'upper'
        arrow_h = 'right'
        mod_v = 'top'
        logscale_j = True
        

    
        
    J_max = len(o_s)*len(dists)
    

    for tick in ticks:

        mods = [B20.tk[tick].cv.cv.cv_v[vol].cv_p[p][o][q].cv_d[dist] for dist in dists for o in o_s]


        fig_U_u_key = plot_fns.rc_plot_setup(name=f"{tick}_U_{u_key}_hat_epsilon_G{p}o{q}-dist", ncols=J_max, nrows=I_max)


        

        if u_key=='u' and incl_N_ax2:
            fig_U_u_key['ax2'] = [[fig_U_u_key['ax'][i][j].twinx() for j in range(J_max)] for i in range(1)] #range(I_max)]
            for i in range(len(fig_U_u_key['ax2'])):
                for j in range(len(fig_U_u_key['ax2'][i])):
                    fig_U_u_key['ax2'][i][j].tick_params(left=False, right=False, top=False, bottom=False,
                                                    labelleft=False, labelright=False, labeltop=False, labelbottom=False)

            for j in range(len(fig_U_u_key['ax2'][i])):
                fig_U_u_key['ax'][0][j].tick_params(right=False,
                                                        labelleft=False, labelright=False, labeltop=False, labelbottom=False)
                fig_U_u_key['ax2'][0][j].tick_params(right=True)
                fig_U_u_key['ax2'][0][j].yaxis.set_label_position("right")
            fig_U_u_key['ax2'][0][-1].tick_params(labelright=True)
            fig_U_u_key['ax'][0][0].tick_params(labelleft=True)
        

        for j in range(J_max):
            o = mods[j]._o
            dist = mods[j]._dist

            plot_bool = {tick: (np.abs(mods[j]._tail_fit[f'{u_key}_fine']['lambda'][f'{u_key}'][0,:])>=xlim[0])*(np.abs(mods[j]._tail_fit[f'{u_key}_fine']['lambda'][f'{u_key}'][0,:])<=xlim[1])}
            

            for k in range(2):

                u_key_x = np.abs(mods[j]._tail_fit[f'{u_key}_fine']['lambda'][f'{u_key}'][k,plot_bool[tick]])
                
                if u_key=='u':
                    fig_U_u_key['ax'][0][j].plot(u_key_x, mods[j]._tail_fit[f'{u_key}_fine']['lambda']['N'][k,plot_bool[tick]]/T[tick], lw=1, color=cols_tail[0][k])
                else:
                    fig_U_u_key['ax'][0][j].plot(u_key_x, np.abs(mods[j]._tail_fit[f'{u_key}_fine']['lambda'][f'u'][k,plot_bool[tick]]), lw=1, color=cols_tail[0][k])
                
                fig_U_u_key['ax'][I_max-4][j].plot(u_key_x, mods[j]._tail_fit[f'{u_key}_fine']['M_stat']['mean'][k,plot_bool[tick]], lw=1, color=cols_tail[0][k])
                    
                for i in range(2):
                    fig_U_u_key['ax'][I_max-3+i][j].plot(u_key_x, mods[j]._tail_fit[f'{u_key}_fine']['gp_fit']['est'][k,plot_bool[tick],2*i], lw=1, color=cols_tail[0][k])
                    fig_U_u_key['ax'][I_max-3+i][j].fill_between(x=u_key_x, y1=mods[j]._tail_fit[f'{u_key}_fine']['gp_fit']['est'][k,plot_bool[tick],2*i]-mods[j]._tail_fit[f'{u_key}_fine']['gp_fit']['se'][k,plot_bool[tick],2*i], 
                                                            y2=mods[j]._tail_fit[f'{u_key}_fine']['gp_fit']['est'][k,plot_bool[tick],2*i]+mods[j]._tail_fit[f'{u_key}_fine']['gp_fit']['se'][k,plot_bool[tick],2*i], 
                                                            color=cols_tail[0][k], alpha=0.2)
                    

                fig_U_u_key['ax'][I_max-1][j].plot(u_key_x, -2*mods[j]._tail_fit[f'{u_key}_fine']['gp_fit']['eval']['log_L'][k,plot_bool[tick]]/mods[j]._tail_fit[f'{u_key}_fine']['lambda']['N'][k,plot_bool[tick]], lw=1, color=cols_tail[0][k])


            for i in logscale_i:
                fig_U_u_key['ax'][i][j].set_yscale('log')
                if u_key=='u' and incl_N_ax2 and i==0:
                    fig_U_u_key['ax2'][i][j].set_yscale('log')
            if logscale_j:
                for i in range(I_max):
                    fig_U_u_key['ax'][i][j].set_xscale('log')
                    if u_key=='u' and incl_N_ax2 and i==0:
                        fig_U_u_key['ax2'][i][j].set_xscale('log')

            fig_U_u_key['ax'][0][j].tick_params(labeltop=False)
            if u_key=='u' and incl_N_ax2:
                fig_U_u_key['ax2'][0][j].tick_params(labeltop=False, top=bool(1-tick_top))

        

            for i in range(I_max):
                if u_key=='u':
                    fig_U_u_key['ax'][i][j].set_xlim(np.flip(xlim))
                else:
                    fig_U_u_key['ax'][i][j].set_xlim(xlim)
                if xticks is not None:
                    fig_U_u_key['ax'][i][j].set_xticks(xticks)

            if u_key=='u':
                fig_U_u_key['ax'][-1][j].set_xlabel(f"$|u^{{\\leftrightharpoons}}|$")
            else:
                fig_U_u_key['ax'][-1][j].set_xlabel(f"$a_{{u}}$")

            if tick_top:
                fig_U_u_key['ax'][0][j].set_xlabel(f"$\\hat{{\\epsilon}}_{{t}} | {cvol_fns.calc_cv_symb(vol=vol, o=mods[j]._o, dist=mods[j]._dist)}$ [{tick.upper()}]")
                fig_U_u_key['ax'][0][j].xaxis.set_label_position("top")
                fig_U_u_key['ax'][0][j].tick_params(top=False, labeltop=False)


        ax_i_j_ylims = np.array([[fig_U_u_key['ax'][i][j].get_ylim() for j in range(J_max)] for i in range(I_max)])
        ax_i_ylim = np.transpose([np.min(ax_i_j_ylims[:,:,0], axis=1), np.max(ax_i_j_ylims[:,:,1], axis=1)])

        for i in range(I_max):
            fig_U_u_key['ax'][i][-1].tick_params(labelright=False)
            for j in range(J_max):
                fig_U_u_key['ax'][i][j].set_ylim(ax_i_ylim[i,:])
                if u_key=='u' and incl_N_ax2 and i==0:
                    fig_U_u_key['ax2'][i][j].set_ylim(ax_i_ylim[i,:]*T[tick])

        if u_key=='u':
            if incl_N_ax2:
                fig_U_u_key['ax2'][0][-1].tick_params(labelright=True)
                fig_U_u_key['ax2'][0][-1].set_ylabel(f"${GR.pltsbv('Delta')}{{{GR.pltsbv('N')}_{{\\epsilon}}}}$")
            fig_U_u_key['ax'][0][0].set_ylabel(f"$\\hat{{{GR.pltsbv('a')}}}_{{\\pi, \\epsilon}}$")
            
        else:
            fig_U_u_key['ax'][0][0].set_ylabel(f"$\\{plot_fns.calc_tex_vect_fn()}{{|u_{{\\epsilon}}|}}$")

        log_L_symb = '\\ell'
        fig_U_u_key['ax'][I_max-4][0].set_ylabel(f"$\\{plot_fns.calc_tex_vect_fn()}{{\\left<\\mathrm{{M}}_{{\\epsilon}}\\right>}}$")
        fig_U_u_key['ax'][I_max-3][0].set_ylabel(f"${misc_fns.capp_tex_fn(GR.pltsbv('xi'), 'hat')}_{{\\epsilon}}$")
        fig_U_u_key['ax'][I_max-2][0].set_ylabel(f"${misc_fns.capp_tex_fn(GR.pltsbv('varsigma'), 'hat')}_{{\\epsilon}}$")
        #fig_U_u_key['ax'][I_max-1][0].set_ylabel(f'$\\ell^{{\\leftrightharpoons}}/N_{{T}}^{{\\leftrightharpoons}}$')
        fig_U_u_key['ax'][I_max-1][0].set_ylabel(f"$-2\\hat{{\\{plot_fns.calc_tex_vect_fn()}{{\\ell}}}}_{{M, \\epsilon}}/{GR.pltsbv('Delta')}{{{GR.pltsbv('N')}_{{\\epsilon}}}}$")
        #fig_U_u_key['ax'][I_max-1][0].set_ylabel(f"$-2{misc_fns.capp_tex_fn(plot_fns.capp_tex_vect(log_L_symb), 'hat')}_{{M}}/{GR.pltsbv('N')}_{{T}}$")



        xy_box = [hwks_fns.annotate_model_Hawkes(ax=fig_U_u_key['ax'][0][j], 
                                            tick=tick, series=f"$\\hat{{\\epsilon}}_{{t}} | {cvol_fns.calc_cv_symb(vol=vol, o=mods[j]._o, dist=mods[j]._dist)}$", plot_tick=bool(1-tick_top), 
                                            str_u=f"$\\Delta{{T}}={plot_fns.capp_tex_num(T[tick])}$", plot_u=(j==J_max-1),
                                            an_h='right', an_v=mod_v, flip_x0ry0t=True,
                                            an_yi=0.13, an_y0=0.04, an_mode=fstmp.get_an_mode()) for j in range(len(fig_U_u_key['ax'][0]))]


        if split_arrow_legend:
            for k in range(2):
                fig_U_u_key['ax'][0][k].legend([fig_U_u_key['ax'][0][0].get_children()[k]],
                                                #[f"$N_{{T}}^{{{arrow}}}$" for arrow in arrows],
                                                [[f"${arrow}$" for arrow in arrows][k]],
                                                loc=f"{arrow_v} {arrow_h}",
                                                bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box[0]], along_x=False, along_y=True),
                                                fontsize=an_params['an_fs'])

        else:
            fig_U_u_key['ax'][0][0].legend([fig_U_u_key['ax'][0][0].get_children()[k] for k in range(2)],
                                                #[f"$N_{{T}}^{{{arrow}}}$" for arrow in arrows],
                                                [f"${arrow}$" for arrow in arrows],
                                                loc=f"{arrow_v} {arrow_h}",
                                                bbox_to_anchor=plot_fns.calc_bbox(xy_boxes=[xy_box[0]], along_x=False, along_y=True),
                                                fontsize=an_params['an_fs'])
        


        

        
        


        fig_U_u_key['fig'].set_size_inches(width,height)
        fig_U_u_key['fig'].tight_layout()

        fstmp.fig_export(fig_U_u_key, height=height, width=width)

    #return fig_U_u_key

In [ ]:
plot_U_u_key_cv(ticks=B20._ticks, u_key='u', incl_N_ax2=True, xticks=None, width=8.5, height=4.5)

In [ ]:
plot_U_u_key_cv(ticks=B20._ticks, u_key='a_u', incl_N_ax2=True, xticks=None, width=8.5, height=4.5)

In [ ]:
tabs = ['r_stat', 'exceed', 'uv_flatx_ticks', 'cv_flats_ticks', 'ucv_LR_ticks', 'ucv_PD_ticks']
tabdir = fstmp.calc_dir(field=['exports', 'tables'])

for pretab in ['ftab', 'tab']:
   fstmp.print_str_pw(print_str=[open(f"{tabdir}{pretab}_{tab}.txt", "r").read() for tab in tabs], do_print=None, do_write=True, write_mode='w', write_address=None, 
                dir=tabdir,
                file_ext=".sty",
                file_nm=f"{fstmp.calc_C_str(C_str_n_post='')['n']}_{pretab}")